# Capstone Project - Week 1 

## Clustering Neighborhoods from Portugal

### Introduction

In this notebook will be executed an exercise where the goal is finding the similarity between places in two closed districts in Portugal **Lisbon** and **Setubal**

### Business Problem

One of the most popular places in Portugal despite of Algarve (south of Portugal) is Lisbon, it's quite often tourists aren't attracted by other places.
There is a district very close to Lisbon called Setubal where you can find attractive things too. The purpose of this exercise is find some similarity between the places in this two districts.

### Data Collection
There are a few sources online that contains information from Portugal districts, counties and boroughs. The most complete list I found was on this website  https://sites.google.com/site/codigosdasfreguesiasdeportugal/lista
* One of the requirements for this exercise it's the **Latitude** and **Longitude** of that specific borough in order to get venues from a specific place

### Foursquare API
To get information about a specific boroughs I'll use Foursquare API. Foursquare is a location data provider where you can see points of interest in a specific area. This points of interest can be restaurants, bars, shops, etc.

### Methodology


### Import the libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Import dataset from borough

In [6]:
csv = r'C:\Users\diogo\Desktop\Coursera\IBM Data Science Certification\ML - Theory Notebooks\Datasets\Portugal_freguesias.csv'

#### The csv has UTF-8 characters, You could use 

```yourdf = pd.read_csv(csv, encoding = A)```

where A could be 'utf-8' or 'utf8'

Otherwise, if not doesn't work, you could use the following code:
``` data = read_csv(filename, encoding = "ISO-8859-1" ```

In [7]:
df = pd.read_csv(csv, encoding = 'utf8')

In [8]:
df.head()

,Distrito;Concelho;FREGUESIA;CÓDIGO
0,AVEIRO;AGUEDA;AGADÃO;10101
1,AVEIRO;AGUEDA;AGUADA DE BAIXO;10102
2,AVEIRO;AGUEDA;AGUADA DE CIMA;10103
3,AVEIRO;AGUEDA;AGUEDA;10104
4,AVEIRO;AGUEDA;BARRO;10105


In [9]:
df.shape

(4289, 1)

#### After the import of the dataframe we can see the df only has 1 column.
In fact, the values are 4 distinct columns combined into 1 column splitted by ";"

The next step is to split the values into to their respective columns, the *n* argument from split function is the number of max separations to make in a single string, where -1 is all.

We want to separate all the values from the column, we could use -1. In fact, n =  -1 or n = 3 it's the same because we 
have "3 separators - ; "

In [10]:
aux = df['Distrito;Concelho;FREGUESIA;CÓDIGO']
df[['District','County','Borough', 'Code']] =  aux.str.split(";", n=-1, expand=True)
df

,Distrito;Concelho;FREGUESIA;CÓDIGO,District,County,Borough,Code
0,AVEIRO;AGUEDA;AGADÃO;10101,AVEIRO,AGUEDA,AGADÃO,10101
1,AVEIRO;AGUEDA;AGUADA DE BAIXO;10102,AVEIRO,AGUEDA,AGUADA DE BAIXO,10102
2,AVEIRO;AGUEDA;AGUADA DE CIMA;10103,AVEIRO,AGUEDA,AGUADA DE CIMA,10103
3,AVEIRO;AGUEDA;AGUEDA;10104,AVEIRO,AGUEDA,AGUEDA,10104
4,AVEIRO;AGUEDA;BARRO;10105,AVEIRO,AGUEDA,BARRO,10105
...,...,...,...,...,...
4284,FUNCHAL;SANTANA;S. ROQUE DO FAIAL;221005,FUNCHAL,SANTANA,S. ROQUE DO FAIAL,221005
4285,FUNCHAL;SANTANA;ILHA;221006,FUNCHAL,SANTANA,ILHA,221006
4286,FUNCHAL;S. VICENTE;BOA VENTURA;221101,FUNCHAL,S. VICENTE,BOA VENTURA,221101
4287,FUNCHAL;S. VICENTE;PONTA DELGADA;221102,FUNCHAL,S. VICENTE,PONTA DELGADA,221102


#### Extract only the interest columns

In [11]:
borough = df.iloc[:,1:]

In [12]:
borough.shape

(4289, 4)

In [13]:
for column in borough.columns:
    borough[column] = df[column].str.title()

In [14]:
borough.head()

,District,County,Borough,Code
0,Aveiro,Agueda,Agadão,10101
1,Aveiro,Agueda,Aguada De Baixo,10102
2,Aveiro,Agueda,Aguada De Cima,10103
3,Aveiro,Agueda,Agueda,10104
4,Aveiro,Agueda,Barro,10105


### The next step is getting the latitudes and longitudes of our dataset. 
We will use **Geopy** to calculate the coordinates where the ``user_agent = foursquare_agent ``

In [15]:
stb = 'Feijó, Setubal, Portugal'
lisb = 'Lisboa, Portugal'

geolocator = Nominatim(user_agent="foursquare_agent")
location_stb = geolocator.geocode(stb)
latitude_stb = location_stb.latitude
longitude_stb = location_stb.longitude

location_lisb = geolocator.geocode(lisb)
latitude_lisb = location_lisb.latitude
longitude_lisb = location_lisb.longitude
print('Setúbal Coordinates: Latitude: {}, Longitude: {}\nLisboa Coordinates: Latitude: {}, Longitude: {}'.format(latitude_stb,longitude_stb,latitude_lisb,longitude_lisb))

Setúbal Coordinates: Latitude: 38.6529791, Longitude: -9.1665562
Lisboa Coordinates: Latitude: 38.7077507, Longitude: -9.1365919


### Best combination

To get the most accurate latitude and longitude, we need to input the most complete string as possible in the geolocator. The best combination would be: District, County, Borough

In [16]:
df2 = borough.iloc[:,:3]

In [17]:
df2.head()

,District,County,Borough
0,Aveiro,Agueda,Agadão
1,Aveiro,Agueda,Aguada De Baixo
2,Aveiro,Agueda,Aguada De Cima
3,Aveiro,Agueda,Agueda
4,Aveiro,Agueda,Barro


In [18]:
df2['Borough'] = df2['Borough'].astype(str)
df2['Borough'] = df2['Borough'].str.replace('S. ','São ')
df2['Borough'] = df2['Borough'].str.replace('(Extinta)', '')
df2['Borough'] = df2['Borough'].str.replace('N. ','Nossa ')

In [19]:
df2['Borough'] = df2['Borough'].replace('Airão (São João Baptista)','São João Batista de Airão')
df2['Borough'] = df2['Borough'].str.replace('N. ','Nossa ')
df2['Borough'] = df2['Borough'].replace('Barrosas (Santa Eulalia) ()','Barrosas (Santa Eulalia)')

#### Sometimes Geolocator don't find some places, so we had to remove those from the dataset

In [20]:
#df2['Borough'].drop('São Pedro De Serracenos', axis = 0)
index = df2[df2['Borough'] == 'São Pedro De Serracenos'].index
df2.drop(index, inplace=True)
df2.shape

(4288, 3)

In [21]:
index2 = df2[df2['Borough'] == 'Monte Perobolço'].index
df2.drop(index2, inplace=True)
df2.shape

(4287, 3)

In [22]:
index3 = df2[df2['Borough'] == 'Souro Pires'].index
df2.drop(index3, inplace=True)
df2.shape

(4286, 3)

In [23]:
index3 = df2[df2['Borough'] == 'Vila Chão Do Marão'].index
df2.drop(index3, inplace=True)
df2.shape

(4285, 3)

### Notes
Sometimes geopy throws an exception when is calculating the latitudes and longitudes from the places, the strategy I've used was to split the "main dataset" into smaller ones, calculate the specific coordinates and in the end merge them all into one.

###  Beja

In [40]:
districts = ['Beja']
filter1 = df2["District"].isin(districts)
df_loc1 = df2[filter1]
df_loc1.shape

(100, 3)

In [41]:
arr_df_loc1 = df_loc1.values.tolist()

In [42]:
lat_df_loc1 = []
long_df_loc2 = []
for d1 in arr_df_loc1:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc1.append(dsq.latitude)
    long_df_loc2.append(dsq.longitude)

Beja,Aljustrel,Aljustrel,
Latitude:37.89032505
Longitude:-8.196355048795365

Beja,Aljustrel,Ervidel,
Latitude:37.9657417
Longitude:-8.086231

Beja,Aljustrel,Messejana,
Latitude:37.8344513
Longitude:-8.2448952

Beja,Aljustrel,São João De Negrilhos,
Latitude:37.9480121
Longitude:-8.2038778

Beja,Aljustrel,Rio De Moinhos,
Latitude:41.1113744
Longitude:-8.2649364

Beja,Almodovar,Almodovar,
Latitude:37.47508055
Longitude:-8.070988093407168

Beja,Almodovar,Gomes Aires,
Latitude:37.48956285
Longitude:-8.221773019830039

Beja,Almodovar,Rosario,
Latitude:-32.9595004
Longitude:-60.6615415

Beja,Almodovar,Santa Clara-A-Nova,
Latitude:37.4651003
Longitude:-8.13730982575689

Beja,Almodovar,Santa Cruz,
Latitude:28.469648
Longitude:-16.2540884

Beja,Almodovar,São Barnabe,
Latitude:37.3576114
Longitude:-8.1646094

Beja,Almodovar,Senhora Da Graça De Padrões,
Latitude:37.5496713
Longitude:-7.955306361050063

Beja,Almodovar,Aldeia Dos Fernandes,
Latitude:37.571207
Longitude:-8.1707493

Beja,Alvito,Alvito

In [43]:
df_loc1['Latitude'] = lat_df_loc1
df_loc1['Longitude'] = long_df_loc2

<ipython-input-43-ada973c6d578>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc1['Latitude'] = lat_df_loc1
<ipython-input-43-ada973c6d578>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc1['Longitude'] = long_df_loc2


In [44]:
df_loc1.head()

,District,County,Borough,Latitude,Longitude
208,Beja,Aljustrel,Aljustrel,37.890325,-8.196355
209,Beja,Aljustrel,Ervidel,37.965742,-8.086231
210,Beja,Aljustrel,Messejana,37.834451,-8.244895
211,Beja,Aljustrel,São João De Negrilhos,37.948012,-8.203878
212,Beja,Aljustrel,Rio De Moinhos,41.111374,-8.264936


In [45]:
df3 = df2.copy()
df3['District'] = df3['District'].str.replace('C Branco','Castelo Branco')
districts2 = ['Castelo Branco','Coimbra','Evora','Faro','Guarda','Leiria']
filter2 = df3["District"].isin(districts2)
df_loc2 = df3[filter2]

In [47]:
arr_df_loc2 = df_loc2.values.tolist()
lat_df_loc2 = []
long_df_loc22 = []
for d1 in arr_df_loc2:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc2.append(dsq.latitude)
    long_df_loc22.append(dsq.longitude)

Castelo Branco,Belmonte,Belmonte,
Latitude:39.5592831
Longitude:-2.7038117

Castelo Branco,Belmonte,Caria,
Latitude:40.2984643
Longitude:-7.3460742

Castelo Branco,Belmonte,Colmeal Da Torre,
Latitude:40.3844721
Longitude:-7.3332355

Castelo Branco,Belmonte,Inguias,
Latitude:40.3255461
Longitude:-7.2963116

Castelo Branco,Belmonte,Maçainhas,
Latitude:40.5326618
Longitude:-7.30695402389714

Castelo Branco,Castelo Branco,Alcains,
Latitude:39.9151369
Longitude:-7.4614654

Castelo Branco,Castelo Branco,Almaceda,
Latitude:40.0071616
Longitude:-7.6616735

Castelo Branco,Castelo Branco,Benquerenças,
Latitude:39.7918605
Longitude:-7.5833411

Castelo Branco,Castelo Branco,Cafede,
Latitude:39.89581285
Longitude:-7.525242827464164

Castelo Branco,Castelo Branco,Castelo Branco,
Latitude:39.97675825
Longitude:-7.446059929966704

Castelo Branco,Castelo Branco,Cebolais De Cima,
Latitude:39.7544502
Longitude:-7.55305475167987

Castelo Branco,Castelo Branco,Escalos De Baixo,
Latitude:39.8593294
Longitud

Castelo Branco,Idanha-A-Nova,Monfortinho,
Latitude:40.0033288
Longitude:-6.9164657

Castelo Branco,Idanha-A-Nova,Monsanto,
Latitude:38.0260319
Longitude:-122.0555188

Castelo Branco,Idanha-A-Nova,Oledo,
Latitude:39.9718181
Longitude:-7.3002672

Castelo Branco,Idanha-A-Nova,Penha Garcia,
Latitude:40.0422569
Longitude:-7.0163521

Castelo Branco,Idanha-A-Nova,Proença-A-Velha,
Latitude:40.0258401
Longitude:-7.2411462

Castelo Branco,Idanha-A-Nova,Rosmaninhal,
Latitude:39.7281391
Longitude:-7.090735

Castelo Branco,Idanha-A-Nova,Salvaterra Do Extremo,
Latitude:39.8837702
Longitude:-6.9143475

Castelo Branco,Idanha-A-Nova,São Miguel De Acha,
Latitude:42.8653505
Longitude:-2.7053329

Castelo Branco,Idanha-A-Nova,Segura,
Latitude:43.0090019
Longitude:-2.2530011

Castelo Branco,Idanha-A-Nova,Toulões,
Latitude:39.9217991
Longitude:-7.0335689

Castelo Branco,Idanha-A-Nova,Zebreira,
Latitude:39.845592
Longitude:-7.0703366

Castelo Branco,Oleiros,Alvaro,
Latitude:39.9755368
Longitude:-7.9653493

Ca

Coimbra,Coimbra,Arzila,
Latitude:35.461928
Longitude:-6.036545

Coimbra,Coimbra,Assafarge,
Latitude:40.1529005
Longitude:-8.434368102323866

Coimbra,Coimbra,Botão,
Latitude:40.32154675
Longitude:-8.395608387163442

Coimbra,Coimbra,Brasfemes,
Latitude:40.2687247
Longitude:-8.39460513284935

Coimbra,Coimbra,Castelo Viegas,
Latitude:40.16601205
Longitude:-8.409124376149265

Coimbra,Coimbra,Ceira,
Latitude:40.18033065
Longitude:-8.380794131784818

Coimbra,Coimbra,Cernache,
Latitude:40.1358462
Longitude:-8.462225474996666

Coimbra,Coimbra,Eiras,
Latitude:40.2482161
Longitude:-8.415478

Coimbra,Coimbra,Lamarosa,
Latitude:39.528698
Longitude:-8.4689558

Coimbra,Coimbra,Ribeira De Frades,
Latitude:40.2024491
Longitude:-8.489408

Coimbra,Coimbra,Santa Clara,
Latitude:37.2333253
Longitude:-121.6846349

Coimbra,Coimbra,Coimbra (Santa Cruz),
Latitude:40.2295975
Longitude:-8.448895850997395

Coimbra,Coimbra,Santo Antonio Dos Olivais,
Latitude:40.2081754
Longitude:-8.392190630655808

Coimbra,Coimbra

Coimbra,Oliveira Do Hospital,São Paio De Gramaços,
Latitude:40.3523882
Longitude:-7.8421262

Coimbra,Oliveira Do Hospital,São Sebastião Da Feira,
Latitude:40.3131362
Longitude:-7.8636099970828495

Coimbra,Oliveira Do Hospital,Seixo Da Beira,
Latitude:40.4589892
Longitude:-7.8451945

Coimbra,Oliveira Do Hospital,Travanca De Lagos,
Latitude:40.3848088
Longitude:-7.8824626

Coimbra,Oliveira Do Hospital,Vila Pouca Da Beira,
Latitude:40.3074438
Longitude:-7.9199282

Coimbra,Oliveira Do Hospital,Vila Franca Da Beira,
Latitude:40.43583835
Longitude:-7.873595857644707

Coimbra,Pampilhosa Da Serra,Cabril,
Latitude:40.09298735
Longitude:-7.889617038132942

Coimbra,Pampilhosa Da Serra,Dornelas Do Zezere,
Latitude:40.103191
Longitude:-7.7455576

Coimbra,Pampilhosa Da Serra,Fajão,
Latitude:40.1641697
Longitude:-7.884380388386383

Coimbra,Pampilhosa Da Serra,Janeiro De Baixo,
Latitude:40.0466701
Longitude:-7.8046921

Coimbra,Pampilhosa Da Serra,Machio,
Latitude:40.00270785
Longitude:-7.9854126318340

Evora,Evora,São Miguel De Machede,
Latitude:38.6501035
Longitude:-7.753275318707761

Evora,Evora,São Pedro (),
Latitude:-22.5512765
Longitude:-47.9072359

Evora,Evora,São Vicente Do Pigeiro,
Latitude:38.44345475
Longitude:-7.643664446794633

Evora,Evora,São (),
Latitude:55.833657349999996
Longitude:37.52580863361071

Evora,Evora,Torre De Coelheiros,
Latitude:38.438544300000004
Longitude:-7.876551396794428

Evora,Evora,São Sebastião Da Giesteira,
Latitude:38.59492085
Longitude:-8.107816623128262

Evora,Evora,Canaviais,
Latitude:38.618829500000004
Longitude:-7.897231494281812

Evora,Evora,Nossa Senhora De Guadalupe,
Latitude:38.58416295
Longitude:-7.998743550543369

Evora,Evora,Baceló,
Latitude:38.6015924
Longitude:-7.926999507919709

Evora,Evora,Horta Das Figueiras,
Latitude:38.53708665
Longitude:-7.905206715533277

Evora,Evora,Malagueira,
Latitude:38.5748285
Longitude:-7.9268848

Evora,Evora,São E São Pedro,
Latitude:-22.5512765
Longitude:-47.9072359

Evora,Evora,Senhora Da Saude,
Lati

Faro,Portimão,Portimão,
Latitude:37.19420955
Longitude:-8.585577395168343

Faro,S. Bras De Alportel,São Bras De Alportel,
Latitude:37.1529622
Longitude:-7.8879138

Faro,Silves,Alcantarilha,
Latitude:37.1601429
Longitude:-8.357123859363359

Faro,Silves,Algoz,
Latitude:37.1679337
Longitude:-8.3032268

Faro,Silves,Armação De Pera,
Latitude:37.1034062
Longitude:-8.3623645

Faro,Silves,Pera,
Latitude:4.812181
Longitude:100.9797908

Faro,Silves,São Bartolomeu De Messines,
Latitude:37.2560769
Longitude:-8.2869009

Faro,Silves,São Marcos Da Serra,
Latitude:37.360692
Longitude:-8.3769927

Faro,Silves,Silves,
Latitude:37.26398255
Longitude:-8.34873166552471

Faro,Silves,Tunes,
Latitude:33.8439408
Longitude:9.400138

Faro,Tavira,Cachopo,
Latitude:37.3330794
Longitude:-7.8172643

Faro,Tavira,Conceição,
Latitude:-7.5565059
Longitude:-38.5053706

Faro,Tavira,Luz,
Latitude:-23.53512225
Longitude:-46.63539301511077

Faro,Tavira,Santa Catarina Da Fonte Do Bispo,
Latitude:37.19291965
Longitude:-7.772296

Guarda,Figueira De Castelo Rodrigo,Reigada,
Latitude:42.5372685
Longitude:-7.4658805

Guarda,Figueira De Castelo Rodrigo,Vale De Afonsinho,
Latitude:40.910010400000004
Longitude:-7.077124184015529

Guarda,Figueira De Castelo Rodrigo,Vermiosa,
Latitude:40.8205009
Longitude:-6.88158095099544

Guarda,Figueira De Castelo Rodrigo,Vilar De Amargo,
Latitude:40.94629485
Longitude:-7.002231248878662

Guarda,Figueira De Castelo Rodrigo,Vilar Torpim,
Latitude:40.8341995
Longitude:-6.948050816750117

Guarda,Fornos De Algodres,Algodres,
Latitude:40.6456612
Longitude:-7.5167205

Guarda,Fornos De Algodres,Casal Vasco,
Latitude:40.6489937
Longitude:-7.5475285

Guarda,Fornos De Algodres,Cortiço,
Latitude:40.663257
Longitude:-7.4938472

Guarda,Fornos De Algodres,Figueiro Da Granja,
Latitude:40.6331302
Longitude:-7.485718397907414

Guarda,Fornos De Algodres,Fornos De Algodres,
Latitude:40.6579679
Longitude:-7.488088682276229

Guarda,Fornos De Algodres,Fuinhas,
Latitude:40.68136035
Longitude:-7.4349396884

Guarda,Meda,Carvalhal,
Latitude:41.5104
Longitude:-8.6350483

Guarda,Meda,Casteição,
Latitude:40.8790182
Longitude:-7.3273377

Guarda,Meda,Coriscada,
Latitude:40.866979
Longitude:-7.207948

Guarda,Meda,Fonte Longa,
Latitude:41.2347907
Longitude:-7.2597367

Guarda,Meda,Longroiva,
Latitude:40.9638819
Longitude:-7.2103641

Guarda,Meda,Marialva,
Latitude:-23.4820987
Longitude:-51.795788

Guarda,Meda,Meda,
Latitude:45.6625046
Longitude:9.1567532

Guarda,Meda,Outeiro De Gatos,
Latitude:40.9522896
Longitude:-7.3028169

Guarda,Meda,Pai Penela,
Latitude:40.9032034
Longitude:-7.2975459

Guarda,Meda,Poço Do Canto,
Latitude:40.9988039
Longitude:-7.2680092

Guarda,Meda,Prova,
Latitude:40.8857873
Longitude:-8.1411094

Guarda,Meda,Rabaçal,
Latitude:40.8545347
Longitude:-7.2577517

Guarda,Meda,Ranhados,
Latitude:40.996662799999996
Longitude:-7.336957159418754

Guarda,Meda,Vale Flor,
Latitude:40.9130761
Longitude:-7.2822335

Guarda,Pinhel,Alverca Da Beira,
Latitude:40.7022917
Longitude:-7.2165758038648

Guarda,Trancoso,Guilheiro,
Latitude:40.8986863
Longitude:-7.4147245

Guarda,Trancoso,Moimentinha,
Latitude:40.7448375
Longitude:-7.2260074

Guarda,Trancoso,Moreira De Rei,
Latitude:40.8304557
Longitude:-7.321562

Guarda,Trancoso,Palhais,
Latitude:40.8159436
Longitude:-7.4315883

Guarda,Trancoso,Povoa Do Concelho,
Latitude:40.7613538
Longitude:-7.2671223

Guarda,Trancoso,Reboleiro,
Latitude:40.8312818
Longitude:-7.4210896

Guarda,Trancoso,Rio De Mel,
Latitude:40.8102092
Longitude:-7.3940281

Guarda,Trancoso,Trancoso (Santa Maria),
Latitude:40.7832755
Longitude:-7.397394865206497

Guarda,Trancoso,Trancoso (São Pedro),
Latitude:40.779067
Longitude:-7.330360171894824

Guarda,Trancoso,Sebadelhe Da Serra,
Latitude:40.8726592
Longitude:-7.36044687349896

Guarda,Trancoso,Souto Maior,
Latitude:40.796536
Longitude:-7.310360684680561

Guarda,Trancoso,Tamanhos,
Latitude:40.7615925
Longitude:-7.3156289

Guarda,Trancoso,Terrenho,
Latitude:40.86672995
Longitude:-7.358682895252336

Guarda,Trancoso,Tor

Leiria,Leiria,Arrabal,
Latitude:39.6981619
Longitude:-8.73171018019886

Leiria,Leiria,Azoia,
Latitude:39.70273065
Longitude:-8.846067662673374

Leiria,Leiria,Barosa,
Latitude:39.75771735
Longitude:-8.85548098961846

Leiria,Leiria,Barreira,
Latitude:-4.2883361
Longitude:-38.641094

Leiria,Leiria,Boa Vista,
Latitude:2.8208478
Longitude:-60.6719582

Leiria,Leiria,Caranguejeira,
Latitude:39.7452346
Longitude:-8.705663807293986

Leiria,Leiria,Carvide,
Latitude:39.85638539999999
Longitude:-8.902666983294209

Leiria,Leiria,Coimbrão,
Latitude:39.921387499999994
Longitude:-8.914112797275415

Leiria,Leiria,Colmeias,
Latitude:39.8069215
Longitude:-8.699541348301015

Leiria,Leiria,Cortes,
Latitude:41.9234871
Longitude:-1.4206937

Leiria,Leiria,Leiria,
Latitude:39.7437902
Longitude:-8.8071119

Leiria,Leiria,Maceira,
Latitude:39.6896146
Longitude:-8.897024224508062

Leiria,Leiria,Marrazes,
Latitude:39.7625962
Longitude:-8.8059313

Leiria,Leiria,Milagres,
Latitude:-12.864635
Longitude:-39.861092

Lei

In [48]:
df_loc2['Latitude'] = lat_df_loc2
df_loc2['Longitude'] = long_df_loc22

<ipython-input-48-1728b6b27f94>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc2['Latitude'] = lat_df_loc2
<ipython-input-48-1728b6b27f94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc2['Longitude'] = long_df_loc22


In [49]:
df_loc2.head()

,District,County,Borough,Latitude,Longitude
1126,Castelo Branco,Belmonte,Belmonte,39.559283,-2.703812
1127,Castelo Branco,Belmonte,Caria,40.298464,-7.346074
1128,Castelo Branco,Belmonte,Colmeal Da Torre,40.384472,-7.333235
1129,Castelo Branco,Belmonte,Inguias,40.325546,-7.296312
1130,Castelo Branco,Belmonte,Maçainhas,40.532662,-7.306954


### Porto 

In [50]:
districts3 = ['Porto']
filter3 = df3["District"].isin(districts3)
df_loc3 = df3[filter3]
df_loc3.shape

(392, 3)

In [51]:
index5 = df_loc3[df_loc3['Borough'] == 'Barrosas (Santa Eulalia)'].index
df_loc3.drop(index5, inplace=True)
df_loc3.shape

C:\Users\diogo\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(391, 3)

In [52]:
index6 = df_loc3[df_loc3['Borough'] == 'A Ver-O-Mar'].index
df_loc3.drop(index6, inplace=True)
df_loc3.shape

df_loc3_1 = df_loc3[0:150]
df_loc3_1.shape

(150, 3)

### Split Porto into 4 datasets because geolocator throws an exception

In [53]:
df_loc3_1 = df_loc3[0:100]
df_loc3_1.shape

(100, 3)

In [54]:
df_loc3_2 = df_loc3[101:201]
df_loc3_2.shape

(100, 3)

In [55]:
df_loc3_3 = df_loc3[201:301]
df_loc3_3.shape

(100, 3)

In [56]:
df_loc3_4 = df_loc3[301:]
df_loc3_4.shape

(89, 3)

In [57]:
arr_df_loc3_1 = df_loc3_1.values.tolist()
lat_df_loc3_1 = []
long_df_loc3_1 = []
for d1 in arr_df_loc3_1:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc3_1.append(dsq.latitude)
    long_df_loc3_1.append(dsq.longitude)

Porto,Amarante,Aboadela,
Latitude:41.2776066
Longitude:-7.9945968

Porto,Amarante,Aboim,
Latitude:41.536473900000004
Longitude:-8.087848376032104

Porto,Amarante,Ansiães,
Latitude:41.2513575
Longitude:-7.9570653

Porto,Amarante,Ataide,
Latitude:-1.8963815
Longitude:-61.5406841

Porto,Amarante,Bustelo,
Latitude:41.7861522
Longitude:-7.4910633

Porto,Amarante,Canadelo,
Latitude:41.3233628
Longitude:-7.9728437

Porto,Amarante,Candemil,
Latitude:41.2473904
Longitude:-7.9681332

Porto,Amarante,Carneiro,
Latitude:-8.474
Longitude:-37.14847075452894

Porto,Amarante,Carvalho De Rei,
Latitude:41.2209103
Longitude:-8.0148461

Porto,Amarante,Cepelos,
Latitude:40.843105
Longitude:-8.344178

Porto,Amarante,Chapa,
Latitude:22.3359668
Longitude:103.8426575

Porto,Amarante,Fregim,
Latitude:41.2702199
Longitude:-8.1170009

Porto,Amarante,Freixo De Baixo,
Latitude:41.2978828
Longitude:-8.1112885

Porto,Amarante,Freixo De Cima,
Latitude:41.3116144
Longitude:-8.1353228

Porto,Amarante,Fridão,
Latitude:41.

In [58]:
df_loc3_1['Latitude'] = lat_df_loc3_1
df_loc3_1['Longitude'] = long_df_loc3_1

<ipython-input-58-1398bcf8d78b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_1['Latitude'] = lat_df_loc3_1
<ipython-input-58-1398bcf8d78b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_1['Longitude'] = long_df_loc3_1


In [59]:
df_loc3_1.head()

,District,County,Borough,Latitude,Longitude
2479,Porto,Amarante,Aboadela,41.277607,-7.994597
2480,Porto,Amarante,Aboim,41.536474,-8.087848
2481,Porto,Amarante,Ansiães,41.251357,-7.957065
2482,Porto,Amarante,Ataide,-1.896381,-61.540684
2483,Porto,Amarante,Bustelo,41.786152,-7.491063


In [60]:
arr_df_loc3_2 = df_loc3_2.values.tolist()
lat_df_loc3_2 = []
long_df_loc3_2 = []
for d1 in arr_df_loc3_2:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc3_2.append(dsq.latitude)
    long_df_loc3_2.append(dsq.longitude)

Porto,Gondomar,São Pedro Da Cova,
Latitude:41.1473685
Longitude:-8.5074154

Porto,Gondomar,Valbom,
Latitude:41.1362281
Longitude:-8.5613183

Porto,Gondomar,Baguim Do Monte (Rio Tinto),
Latitude:41.185524650000005
Longitude:-8.545846824747183

Porto,Lousada,Alvarenga,
Latitude:-19.4175
Longitude:-41.728334

Porto,Lousada,Aveleda,
Latitude:41.2633394
Longitude:-8.6895353

Porto,Lousada,Boim,
Latitude:-2.856879
Longitude:-55.60880537185473

Porto,Lousada,Caide De Rei,
Latitude:41.2665233
Longitude:-8.2216457

Porto,Lousada,Casais,
Latitude:40.2047747
Longitude:-8.4835255

Porto,Lousada,Cernadelo,
Latitude:42.4874269
Longitude:-8.3500195

Porto,Lousada,Covas,
Latitude:42.9199733
Longitude:-8.74966

Porto,Lousada,Cristelos,
Latitude:41.2745805
Longitude:-8.2903482

Porto,Lousada,Figueiras,
Latitude:38.8913933
Longitude:-9.2820277

Porto,Lousada,Lodares,
Latitude:41.2389255
Longitude:-8.2922928

Porto,Lousada,Lustosa,
Latitude:-12.23532295
Longitude:-38.62790453388665

Porto,Lousada,Macieira

In [61]:
df_loc3_2['Latitude'] = lat_df_loc3_2
df_loc3_2['Longitude'] = long_df_loc3_2

<ipython-input-61-71190d58809a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_2['Latitude'] = lat_df_loc3_2
<ipython-input-61-71190d58809a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_2['Longitude'] = long_df_loc3_2


In [62]:
df_loc3_2.head()

,District,County,Borough,Latitude,Longitude
2581,Porto,Gondomar,São Pedro Da Cova,41.147368,-8.507415
2582,Porto,Gondomar,Valbom,41.136228,-8.561318
2583,Porto,Gondomar,Baguim Do Monte (Rio Tinto),41.185525,-8.545847
2584,Porto,Lousada,Alvarenga,-19.417500,-41.728334
2585,Porto,Lousada,Aveleda,41.263339,-8.689535


In [63]:
arr_df_loc3_3 = df_loc3_3.values.tolist()
lat_df_loc3_3 = []
long_df_loc3_3 = []
for d1 in arr_df_loc3_3:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc3_3.append(dsq.latitude)
    long_df_loc3_3.append(dsq.longitude)

Porto,Paços De Ferreira,Sanfins De Ferreira,
Latitude:41.3266496
Longitude:-8.3721821

Porto,Paços De Ferreira,Seroa,
Latitude:41.2641367
Longitude:-8.4349395

Porto,Paredes,Aguiar De Sousa,
Latitude:41.1271547
Longitude:-8.4477733

Porto,Paredes,Astromil,
Latitude:41.1981228
Longitude:-8.4195664

Porto,Paredes,Baltar,
Latitude:41.9509733
Longitude:-7.7157216

Porto,Paredes,Beire,
Latitude:48.9467562
Longitude:11.4038717

Porto,Paredes,Besteiros,
Latitude:37.5261103
Longitude:-7.6689604

Porto,Paredes,Bitarães,
Latitude:41.2256945
Longitude:-8.3147825

Porto,Paredes,Castelões De Cepeda,
Latitude:41.2069945
Longitude:-8.3292543

Porto,Paredes,Cete,
Latitude:41.1782871
Longitude:-8.3649391

Porto,Paredes,Cristelo,
Latitude:41.4785773
Longitude:-8.6965942

Porto,Paredes,Duas Igrejas,
Latitude:41.230974
Longitude:-8.3772602

Porto,Paredes,Gandra,
Latitude:41.1963232
Longitude:-8.4359653

Porto,Paredes,Gondalães,
Latitude:41.2287102
Longitude:-8.3319853

Porto,Paredes,Lordelo,
Latitude:41.3

In [64]:
df_loc3_3['Latitude'] = lat_df_loc3_3
df_loc3_3['Longitude'] = long_df_loc3_3

<ipython-input-64-ec19d336464d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_3['Latitude'] = lat_df_loc3_3
<ipython-input-64-ec19d336464d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_3['Longitude'] = long_df_loc3_3


In [65]:
arr_df_loc3_4 = df_loc3_4.values.tolist()
lat_df_loc3_4 = []
long_df_loc3_4 = []
for d1 in arr_df_loc3_4:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc3_4.append(dsq.latitude)
    long_df_loc3_4.append(dsq.longitude)

Porto,Santo Tirso,Lama,
Latitude:42.5767128
Longitude:9.1717394

Porto,Santo Tirso,Lamelas,
Latitude:41.2875059
Longitude:-8.4744184

Porto,Santo Tirso,Monte Cordova,
Latitude:41.3164414
Longitude:-8.4320306

Porto,Santo Tirso,Muro (),
Latitude:42.5459792
Longitude:8.9147021

Porto,Santo Tirso,Palmeira,
Latitude:-27.9010519
Longitude:-53.3135927

Porto,Santo Tirso,Rebordões,
Latitude:41.3524388
Longitude:-8.4295071

Porto,Santo Tirso,Refojos De Riba De Ave,
Latitude:41.2920945
Longitude:-8.4471657

Porto,Santo Tirso,Reguenga,
Latitude:41.2755959
Longitude:-8.4572825

Porto,Santo Tirso,Roriz,
Latitude:41.5842966
Longitude:-8.592986

Porto,Santo Tirso,Couto (Santa Cristina),
Latitude:41.325741
Longitude:-8.4793533

Porto,Santo Tirso,Bougado (Santiago) (),
Latitude:41.3302367
Longitude:-8.5724918

Porto,Santo Tirso,Santo Tirso,
Latitude:41.3440057
Longitude:-8.4772021

Porto,Santo Tirso,Coronado (São Mamede) (),
Latitude:41.2748271
Longitude:-8.5739705

Porto,Santo Tirso,Negrelos (São Mam

In [66]:
df_loc3_4['Latitude'] = lat_df_loc3_4
df_loc3_4['Longitude'] = long_df_loc3_4

<ipython-input-66-7ca528b1d3f3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_4['Latitude'] = lat_df_loc3_4
<ipython-input-66-7ca528b1d3f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc3_4['Longitude'] = long_df_loc3_4


In [67]:
df_porto = df_loc3_1.append([df_loc3_2,df_loc3_3,df_loc3_4], ignore_index=True)

In [68]:
val = df3.District.unique()
val

array(['Aveiro', 'Beja', 'Braga', 'Braganca', 'Castelo Branco', 'Coimbra',
       'Evora', 'Faro', 'Guarda', 'Leiria', 'Lisboa', 'Portalegre',
       'Porto', 'Santarem', 'Setubal', 'Viana Do Castelo', 'Vila Real',
       'Viseu', 'Angra Do Heroismo', 'Horta', 'Ponta Delgada', 'Funchal'],
      dtype=object)

### Santarem / Setubal

In [69]:
districts4 = ['Santarem', 'Setubal']
filter4 = df2["District"].isin(districts4)
df_loc4 = df2[filter4]
df_loc4.shape

(274, 3)

In [70]:
arr_df_loc4 = df_loc4.values.tolist()
lat_df_loc4 = []
long_df_loc4 = []
for d1 in arr_df_loc4:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc4.append(dsq.latitude)
    long_df_loc4.append(dsq.longitude)

Santarem,Abrantes,Aldeia Do Mato,
Latitude:39.53807085
Longitude:-8.26554685378136

Santarem,Abrantes,Alferrarede,
Latitude:39.491749799999994
Longitude:-8.161656135868824

Santarem,Abrantes,Alvega,
Latitude:39.4308573
Longitude:-8.006832694939439

Santarem,Abrantes,Bemposta,
Latitude:41.3143431
Longitude:-6.5012976

Santarem,Abrantes,Martinchel,
Latitude:39.5370725
Longitude:-8.3127578

Santarem,Abrantes,Mouriscas,
Latitude:39.50524905
Longitude:-8.094088896730788

Santarem,Abrantes,Pego,
Latitude:41.3341528
Longitude:-5.4672646

Santarem,Abrantes,Rio De Moinhos,
Latitude:41.1113744
Longitude:-8.2649364

Santarem,Abrantes,Rossio Ao Sul Do Tejo,
Latitude:39.4357775
Longitude:-8.170740270798532

Santarem,Abrantes,São Facundo,
Latitude:39.3850862
Longitude:-8.059427799741709

Santarem,Abrantes,Abrantes (São João),
Latitude:39.45782695
Longitude:-8.193843559300067

Santarem,Abrantes,São Miguel Do Rio Torto,
Latitude:39.4072645
Longitude:-8.212027692975639

Santarem,Abrantes,Abrantes (São 

Santarem,Santarem,Alcanede,
Latitude:39.41517945
Longitude:-8.828159570703919

Santarem,Santarem,Alcanhões,
Latitude:39.2958027
Longitude:-8.6583378

Santarem,Santarem,Almoster,
Latitude:41.1960574
Longitude:1.1111352

Santarem,Santarem,Amiais De Baixo,
Latitude:39.443308
Longitude:-8.7317219

Santarem,Santarem,Arneiro Das Milhariças,
Latitude:39.394950550000004
Longitude:-8.720129007294434

Santarem,Santarem,Azoia De Baixo,
Latitude:39.28968545
Longitude:-8.714158738989747

Santarem,Santarem,Azoia De Cima,
Latitude:39.3511314
Longitude:-8.737515476793373

Santarem,Santarem,Casevel,
Latitude:39.400220250000004
Longitude:-8.602145618292088

Santarem,Santarem,Santarem (Marvila),
Latitude:39.2234659
Longitude:-8.6871553

Santarem,Santarem,Moçarria,
Latitude:39.271839
Longitude:-8.772165316039967

Santarem,Santarem,Pernes,
Latitude:50.4841033
Longitude:2.410749

Santarem,Santarem,Pombalinho,
Latitude:39.99175945
Longitude:-8.500393220158024

Santarem,Santarem,Povoa Da Isenta,
Latitude:39.2

Setubal,Almada,Laranjeiro,
Latitude:38.6565296
Longitude:-9.1542243

Setubal,Almada,Feijo,
Latitude:-8.1648652
Longitude:-70.3539579

Setubal,Barreiro,Barreiro,
Latitude:38.6340657
Longitude:-9.047856727251068

Setubal,Barreiro,Lavradio,
Latitude:38.66870125
Longitude:-9.05161928703556

Setubal,Barreiro,Palhais,
Latitude:40.8159436
Longitude:-7.4315883

Setubal,Barreiro,Santo Andre,
Latitude:-23.6573395
Longitude:-46.5322504

Setubal,Barreiro,Verderena,
Latitude:38.6528467
Longitude:-9.07154

Setubal,Barreiro,Alto Do Seixalinho,
Latitude:38.6575416
Longitude:-9.0661472

Setubal,Barreiro,Santo Antonio Da Charneca,
Latitude:38.6246943
Longitude:-9.031607

Setubal,Barreiro,Coina,
Latitude:38.5849349
Longitude:-9.051412511507941

Setubal,Grandola,Azinheira Dos Barros E São Mamede Do Sadão,
Latitude:38.1552027
Longitude:-8.3388967

Setubal,Grandola,Grandola,
Latitude:38.1761212
Longitude:-8.5680717

Setubal,Grandola,Melides,
Latitude:38.17937215
Longitude:-8.718121168932608

Setubal,Grandol

In [71]:
df_loc4['Latitude'] = lat_df_loc4
df_loc4['Longitude'] = long_df_loc4

<ipython-input-71-610608ec5be0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc4['Latitude'] = lat_df_loc4
<ipython-input-71-610608ec5be0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc4['Longitude'] = long_df_loc4


### Viana Do Castelo

In [72]:
districts5 = ['Viana Do Castelo']
filter5 = df2["District"].isin(districts5)
df_loc5 = df2[filter5]
df_loc5.shape

(290, 3)

In [73]:
arr_df_loc5 = df_loc5.values.tolist()
lat_df_loc5 = []
long_df_loc5 = []
for d1 in arr_df_loc5:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc5.append(dsq.latitude)
    long_df_loc5.append(dsq.longitude)

Viana Do Castelo,Arcos De Valdevez,Aboim Das Choças,
Latitude:41.9237483
Longitude:-8.456035

Viana Do Castelo,Arcos De Valdevez,Aguiã,
Latitude:41.8845639
Longitude:-8.4433445

Viana Do Castelo,Arcos De Valdevez,Alvora,
Latitude:61.578737399999994
Longitude:5.1258957547275426

Viana Do Castelo,Arcos De Valdevez,Azere,
Latitude:41.8645807
Longitude:-8.3916654

Viana Do Castelo,Arcos De Valdevez,Cabana Maior,
Latitude:41.8707832
Longitude:-8.3518088

Viana Do Castelo,Arcos De Valdevez,Cabreiro,
Latitude:41.9327534
Longitude:-8.4151051

Viana Do Castelo,Arcos De Valdevez,Carralcova,
Latitude:42.649197
Longitude:-7.3474767

Viana Do Castelo,Arcos De Valdevez,Cendufe,
Latitude:41.813005
Longitude:-8.4869147

Viana Do Castelo,Arcos De Valdevez,Couto,
Latitude:41.874892
Longitude:-8.4084898

Viana Do Castelo,Arcos De Valdevez,Eiras,
Latitude:40.2482161
Longitude:-8.415478

Viana Do Castelo,Arcos De Valdevez,Ermelo,
Latitude:52.2858356
Longitude:5.654938589920705

Viana Do Castelo,Arcos De Va

Viana Do Castelo,Monção,Lara,
Latitude:10.2284518
Longitude:-69.8197975

Viana Do Castelo,Monção,Longos Vales,
Latitude:42.0496148
Longitude:-8.4404588

Viana Do Castelo,Monção,Lordelo,
Latitude:41.3725051
Longitude:-8.3787227

Viana Do Castelo,Monção,Luzio,
Latitude:37.2293842
Longitude:-8.51331

Viana Do Castelo,Monção,Mazedo,
Latitude:42.0571577
Longitude:-8.4901852

Viana Do Castelo,Monção,Merufe,
Latitude:42.0297394
Longitude:-8.3967117

Viana Do Castelo,Monção,Messegães,
Latitude:42.0776183
Longitude:-8.3663993

Viana Do Castelo,Monção,Monção,
Latitude:42.0787023
Longitude:-8.4813901

Viana Do Castelo,Monção,Moreira,
Latitude:42.0234185
Longitude:-8.4853967

Viana Do Castelo,Monção,Parada,
Latitude:44.3880703
Longitude:0.4472192

Viana Do Castelo,Monção,Pias,
Latitude:42.0919796
Longitude:-6.9922396

Viana Do Castelo,Monção,Pinheiros,
Latitude:-23.567249
Longitude:-46.7019515

Viana Do Castelo,Monção,Podame,
Latitude:42.0420535
Longitude:-8.369954

Viana Do Castelo,Monção,Portela

Viana Do Castelo,Ponte De Lima,Labrujo,
Latitude:41.8543636
Longitude:-8.541474

Viana Do Castelo,Ponte De Lima,Mato,
Latitude:34.9169043
Longitude:98.2122828

Viana Do Castelo,Ponte De Lima,Moreira Do Lima,
Latitude:41.7865624
Longitude:-8.6247957

Viana Do Castelo,Ponte De Lima,Navio,
Latitude:42.3696294
Longitude:-8.0851946

Viana Do Castelo,Ponte De Lima,Poiares,
Latitude:41.0635353
Longitude:-6.8391868

Viana Do Castelo,Ponte De Lima,Ponte De Lima,
Latitude:41.7675021
Longitude:-8.5830992

Viana Do Castelo,Ponte De Lima,Queijada,
Latitude:41.717528
Longitude:-8.5360624

Viana Do Castelo,Ponte De Lima,Refoios Do Lima,
Latitude:41.7930651
Longitude:-8.5417319

Viana Do Castelo,Ponte De Lima,Rendufe,
Latitude:41.4782335
Longitude:-8.2318882

Viana Do Castelo,Ponte De Lima,Ribeira,
Latitude:42.5805628
Longitude:-9.013685

Viana Do Castelo,Ponte De Lima,Sa,
Latitude:36.638392
Longitude:127.6961188

Viana Do Castelo,Ponte De Lima,Sandiães,
Latitude:42.1106859
Longitude:-7.7577611

Viana

In [74]:
df_loc5['Latitude'] = lat_df_loc5
df_loc5['Longitude'] = long_df_loc5

<ipython-input-74-911b6ce9aa98>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc5['Latitude'] = lat_df_loc5
<ipython-input-74-911b6ce9aa98>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc5['Longitude'] = long_df_loc5


### Lisboa / Portalegre / Vila Real

In [75]:
districts6 = ['Lisboa','Portalegre','Vila Real']
filter6 = df2["District"].isin(districts6)
df_loc6 = df2[filter6]
df_loc6.shape

(589, 3)

In [76]:
arr_df_loc6 = df_loc6.values.tolist()
lat_df_loc6 = []
long_df_loc6 = []
for d1 in arr_df_loc6:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc6.append(dsq.latitude)
    long_df_loc6.append(dsq.longitude)

Lisboa,Alenquer,Abrigada,
Latitude:39.14483655
Longitude:-9.023279351101172

Lisboa,Alenquer,Aldeia Galega Da Merceana,
Latitude:39.0829886
Longitude:-9.1117742

Lisboa,Alenquer,Aldeia Gavinha,
Latitude:39.08948745
Longitude:-9.088724721487635

Lisboa,Alenquer,Cabanas De Torres,
Latitude:39.156263949999996
Longitude:-9.06018891782175

Lisboa,Alenquer,Cadafais,
Latitude:39.0116324
Longitude:-9.0035234

Lisboa,Alenquer,Carnota,
Latitude:42.821959
Longitude:-9.0878032

Lisboa,Alenquer,Meca,
Latitude:49.1196964
Longitude:6.1763552

Lisboa,Alenquer,Olhalvo,
Latitude:39.10061595
Longitude:-9.063868186841672

Lisboa,Alenquer,Ota,
Latitude:-45.1299859
Longitude:169.5248818

Lisboa,Alenquer,Pereiro De Palhacana,
Latitude:39.0432393
Longitude:-9.1106194

Lisboa,Alenquer,Alenquer (Santo Estevão),
Latitude:39.0427614
Longitude:-9.0201159

Lisboa,Alenquer,Alenquer (Triana),
Latitude:39.0661561
Longitude:-8.9939375

Lisboa,Alenquer,Ventosa,
Latitude:42.4042098
Longitude:-2.6263385

Lisboa,Alenquer,V

Lisboa,Loures,Santa Iria De Azoia,
Latitude:38.8443575
Longitude:-9.088739018185784

Lisboa,Loures,Santo Antão Do Tojal,
Latitude:38.8568017
Longitude:-9.159850318024258

Lisboa,Loures,São João Da Talha,
Latitude:38.82544575
Longitude:-9.097862555058231

Lisboa,Loures,São Julião Do Tojal,
Latitude:38.8634153
Longitude:-9.122751715914255

Lisboa,Loures,Unhos,
Latitude:38.8206991
Longitude:-9.119487106715276

Lisboa,Loures,Pontinha (),
Latitude:38.7622108
Longitude:-9.1968418

Lisboa,Loures,Portela,
Latitude:41.3008566
Longitude:-8.5487452

Lisboa,Loures,Famões(),
Latitude:38.7912413
Longitude:-9.2108576

Lisboa,Loures,Ramada(),
Latitude:27.622145
Longitude:-105.7298978

Lisboa,Loures,Bobadela,
Latitude:38.80674275
Longitude:-9.094721545420715

Lisboa,Loures,Prior Velho,
Latitude:38.789594550000004
Longitude:-9.121975085555158

Lisboa,Loures,Santo Antonio Dos Cavaleiros,
Latitude:38.8110542
Longitude:-9.16892660650544

Lisboa,Loures,Olival De Basto(),
Latitude:38.796549549999995
Longitud

Lisboa,Vila Franca De Xira,Vialonga,
Latitude:38.8735511
Longitude:-9.0785504

Lisboa,Vila Franca De Xira,Vila Franca De Xira,
Latitude:38.912368400000005
Longitude:-8.988824041991643

Lisboa,Vila Franca De Xira,Sobralinho,
Latitude:38.9173535
Longitude:-9.0260969

Lisboa,Vila Franca De Xira,Forte Da Casa,
Latitude:38.873491
Longitude:-9.050401822259616

Lisboa,Amadora,Alfragide,
Latitude:38.73366745
Longitude:-9.216004474419574

Lisboa,Amadora,Brandoa,
Latitude:38.7672892
Longitude:-9.217470104291667

Lisboa,Amadora,Buraca,
Latitude:38.7437889
Longitude:-9.2082841

Lisboa,Amadora,Damaia,
Latitude:12.7957158
Longitude:11.5375783

Lisboa,Amadora,Falagueira,
Latitude:38.7613141
Longitude:-9.2259784

Lisboa,Amadora,Mina,
Latitude:26.299386900000002
Longitude:-100.80250458701417

Lisboa,Amadora,Reboleira,
Latitude:38.7523013
Longitude:-9.2240282

Lisboa,Amadora,Venteira,
Latitude:38.746758549999996
Longitude:-9.240497507432714

Lisboa,Amadora,Alfornelos,
Latitude:38.7603326
Longitude:-9.20

Portalegre,Sousel,Santo Amaro,
Latitude:-12.5519686
Longitude:-38.7060448

Portalegre,Sousel,Sousel,
Latitude:38.95224
Longitude:-7.67603

Vila Real,Alijo,Alijo,
Latitude:41.2768619
Longitude:-7.4753795

Vila Real,Alijo,Amieiro,
Latitude:41.279042
Longitude:-7.3963815

Vila Real,Alijo,Carlão,
Latitude:41.31703
Longitude:-7.408736

Vila Real,Alijo,Casal De Loivos,
Latitude:41.2018753
Longitude:-7.5277799

Vila Real,Alijo,Castedo,
Latitude:41.2318928
Longitude:-7.1706999

Vila Real,Alijo,Cotas,
Latitude:41.2184371
Longitude:-7.4851273

Vila Real,Alijo,Favaios,
Latitude:41.2674081
Longitude:-7.5005228

Vila Real,Alijo,Pegarinhos,
Latitude:41.3559603
Longitude:-7.4452731

Vila Real,Alijo,Pinhão,
Latitude:-25.6960895
Longitude:-51.6566366

Vila Real,Alijo,Populo,
Latitude:36.5308664
Longitude:-6.29495

Vila Real,Alijo,Ribalonga,
Latitude:41.2205386
Longitude:-7.3936426

Vila Real,Alijo,Sanfins Do Douro,
Latitude:41.2931814
Longitude:-7.5187469

Vila Real,Alijo,Santa Eugenia,
Latitude:40.382

Vila Real,Montalegre,Ferral,
Latitude:41.7052252
Longitude:-7.9878141

Vila Real,Montalegre,Fervidelas,
Latitude:41.7550675
Longitude:-7.8993661

Vila Real,Montalegre,Fiães Do Rio,
Latitude:41.7780136
Longitude:-7.926292

Vila Real,Montalegre,Gralhas,
Latitude:41.8504336
Longitude:-7.7038253

Vila Real,Montalegre,Meixedo,
Latitude:41.7593271
Longitude:-8.7076436

Vila Real,Montalegre,Meixide,
Latitude:42.8878158
Longitude:-7.9103282

Vila Real,Montalegre,Montalegre,
Latitude:41.8232206
Longitude:-7.7923142

Vila Real,Montalegre,Morgade,
Latitude:41.7567744
Longitude:-7.7464892

Vila Real,Montalegre,Mourilhe,
Latitude:41.836741
Longitude:-7.8417546

Vila Real,Montalegre,Negrões,
Latitude:10.0206654
Longitude:122.97715983111425

Vila Real,Montalegre,Outeiro,
Latitude:41.6824433
Longitude:-6.6009037

Vila Real,Montalegre,Padornelos,
Latitude:41.8614797
Longitude:-7.7548701

Vila Real,Montalegre,Padroso,
Latitude:41.9418847
Longitude:-8.490082

Vila Real,Montalegre,Paradela,
Latitude:42.76

Vila Real,Valpaços,Vales,
Latitude:46.2303063
Longitude:7.6605757

Vila Real,Valpaços,Valpaços,
Latitude:41.6067883
Longitude:-7.3108525

Vila Real,Valpaços,Vassal,
Latitude:41.6167863
Longitude:-7.3566981

Vila Real,Valpaços,Veiga De Lila,
Latitude:41.5229045
Longitude:-7.3250932

Vila Real,Valpaços,Vilarandelo,
Latitude:41.6634306
Longitude:-7.3264197

Vila Real,Vila Pouca De Aguiar,Afonsim,
Latitude:40.7664527
Longitude:-7.6960982

Vila Real,Vila Pouca De Aguiar,Alfarela De Jales,
Latitude:41.4445414
Longitude:-7.5718811

Vila Real,Vila Pouca De Aguiar,Bornes De Aguiar,
Latitude:41.5273642
Longitude:-7.610874

Vila Real,Vila Pouca De Aguiar,Bragado,
Latitude:-35.1155766
Longitude:-60.489771

Vila Real,Vila Pouca De Aguiar,Capeludos,
Latitude:41.6242036
Longitude:-7.6339241

Vila Real,Vila Pouca De Aguiar,Gouvães Da Serra,
Latitude:41.4743674
Longitude:-7.7294524

Vila Real,Vila Pouca De Aguiar,Parada De Monteiros,
Latitude:41.5644686
Longitude:-7.720191

Vila Real,Vila Pouca De Agui

In [77]:
df_loc6['Latitude'] = lat_df_loc6
df_loc6['Longitude'] = long_df_loc6

<ipython-input-77-3ccc3b2621f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc6['Latitude'] = lat_df_loc6
<ipython-input-77-3ccc3b2621f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc6['Longitude'] = long_df_loc6


### Viseu / Bragança / Horta / Angra do Heroismo / Funchal / Ponta Delgada

In [78]:
districts7 = ['Viseu','Braganca','Horta', 'Funchal','Ponta Delgada']
filter7 = df2["District"].isin(districts7)
df_loc7 = df2[filter7]
df_loc7.shape

(838, 3)

In [79]:
arr_df_loc7 = df_loc7.values.tolist()
lat_df_loc7 = []
long_df_loc7 = []
for d1 in arr_df_loc7:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc7.append(dsq.latitude)
    long_df_loc7.append(dsq.longitude)

Braganca,Alfandega Da Fe,Agrobom,
Latitude:41.3981559
Longitude:-6.9150768

Braganca,Alfandega Da Fe,Alfandega Da Fe,
Latitude:41.3419743
Longitude:-6.9611439

Braganca,Alfandega Da Fe,Cerejais,
Latitude:41.2937361
Longitude:-6.9267559

Braganca,Alfandega Da Fe,Eucisia,
Latitude:41.3136306
Longitude:-7.0189685

Braganca,Alfandega Da Fe,Ferradosa,
Latitude:41.2747335
Longitude:-6.9545656

Braganca,Alfandega Da Fe,Gebelim,
Latitude:41.4421268
Longitude:-6.9306362

Braganca,Alfandega Da Fe,Gouveia,
Latitude:40.49773625
Longitude:-7.595845584129108

Braganca,Alfandega Da Fe,Parada,
Latitude:44.3880703
Longitude:0.4472192

Braganca,Alfandega Da Fe,Pombal,
Latitude:-6.7711315
Longitude:-37.7988646

Braganca,Alfandega Da Fe,Saldonha,
Latitude:41.407934
Longitude:-6.8763426

Braganca,Alfandega Da Fe,Sambade,
Latitude:41.4130873
Longitude:-6.9747481

Braganca,Alfandega Da Fe,Sendim Da Ribeira,
Latitude:41.3104746
Longitude:-6.9055808

Braganca,Alfandega Da Fe,Sendim Da Serra,
Latitude:41.294169

Braganca,Macedo De Cavaleiros,Ferreira,
Latitude:37.1723754
Longitude:-3.0362531

Braganca,Macedo De Cavaleiros,Grijo,
Latitude:41.5027019
Longitude:-6.9761157

Braganca,Macedo De Cavaleiros,Lagoa,
Latitude:37.13258125
Longitude:-8.45505094811731

Braganca,Macedo De Cavaleiros,Lamalonga,
Latitude:41.6784995
Longitude:-7.0794812

Braganca,Macedo De Cavaleiros,Lamas,
Latitude:-6.2655949
Longitude:-76.5073652

Braganca,Macedo De Cavaleiros,Lombo,
Latitude:41.4580101
Longitude:-6.846829

Braganca,Macedo De Cavaleiros,Macedo De Cavaleiros,
Latitude:41.5362124
Longitude:-6.9560267

Braganca,Macedo De Cavaleiros,Morais,
Latitude:41.4900476
Longitude:-6.7761265

Braganca,Macedo De Cavaleiros,Murços,
Latitude:41.6796397
Longitude:-6.9827118

Braganca,Macedo De Cavaleiros,Olmos,
Latitude:-33.5020648
Longitude:-63.16653758953701

Braganca,Macedo De Cavaleiros,Peredo,
Latitude:41.4289615
Longitude:-6.8693807

Braganca,Macedo De Cavaleiros,Podence,
Latitude:41.5913228
Longitude:-6.9274602

Braganca

Braganca,Mogadouro,Vilarinho Dos Galegos,
Latitude:41.2660628
Longitude:-6.6256544

Braganca,Torre De Moncorvo,Açoreira,
Latitude:41.1394582
Longitude:-7.0406665

Braganca,Torre De Moncorvo,Adeganha,
Latitude:41.2746383
Longitude:-7.0511914

Braganca,Torre De Moncorvo,Cabeça Boa,
Latitude:41.1891707
Longitude:-7.1495387

Braganca,Torre De Moncorvo,Cardanha,
Latitude:41.2561304
Longitude:-7.0332099

Braganca,Torre De Moncorvo,Carviçais,
Latitude:41.1813234
Longitude:-6.8921364

Braganca,Torre De Moncorvo,Castedo,
Latitude:41.2318928
Longitude:-7.1706999

Braganca,Torre De Moncorvo,Felgar,
Latitude:41.2100878
Longitude:-6.9579042

Braganca,Torre De Moncorvo,Felgueiras,
Latitude:41.3647716
Longitude:-8.1989321

Braganca,Torre De Moncorvo,Horta Da Vilariça,
Latitude:41.23274
Longitude:-7.1218623

Braganca,Torre De Moncorvo,Larinho,
Latitude:41.2051918
Longitude:-7.0125499

Braganca,Torre De Moncorvo,Lousa,
Latitude:38.8898108
Longitude:-9.2059575

Braganca,Torre De Moncorvo,Maçores,
Latitu

Viseu,Castro Daire,Alva,
Latitude:36.8050308
Longitude:-98.6664737

Viseu,Castro Daire,Cabril,
Latitude:40.09298735
Longitude:-7.889617038132942

Viseu,Castro Daire,Castro Daire,
Latitude:40.9014091
Longitude:-7.9311761

Viseu,Castro Daire,Cujo,
Latitude:40.9384781
Longitude:-7.8438379

Viseu,Castro Daire,Ermida,
Latitude:41.2302701
Longitude:-7.745984419137983

Viseu,Castro Daire,Ester,
Latitude:64.8586885
Longitude:-148.09005916666666

Viseu,Castro Daire,Gafanhão,
Latitude:40.89085
Longitude:-8.02867

Viseu,Castro Daire,Gosende,
Latitude:41.0083477
Longitude:-7.9058671

Viseu,Castro Daire,Mamouros,
Latitude:40.8365589
Longitude:-7.9459119

Viseu,Castro Daire,Mezio,
Latitude:40.9841603
Longitude:-7.8916425

Viseu,Castro Daire,Mões,
Latitude:43.1818
Longitude:2.64968

Viseu,Castro Daire,Moledo,
Latitude:39.2967223
Longitude:-9.253666383475323

Viseu,Castro Daire,Monteiras,
Latitude:41.1774533
Longitude:-7.5405507

Viseu,Castro Daire,Moura Morta,
Latitude:41.1968586
Longitude:-7.8345645

Viseu,Nelas,Vilar Seco,
Latitude:41.5251801
Longitude:-6.4029545

Viseu,Nelas,Aguieira,
Latitude:41.753202
Longitude:-8.7538135

Viseu,Nelas,Lapa Do Lobo,
Latitude:40.4746034
Longitude:-7.9162792

Viseu,Nelas,Moreira,
Latitude:42.0234185
Longitude:-8.4853967

Viseu,Oliveira De Frades,Arca,
Latitude:41.7617931
Longitude:-8.5763975

Viseu,Oliveira De Frades,Arcozelo Das Maias,
Latitude:40.7353971
Longitude:-8.2776103

Viseu,Oliveira De Frades,Destriz,
Latitude:40.6508584
Longitude:-8.2820723

Viseu,Oliveira De Frades,Oliveira De Frades,
Latitude:40.7322108
Longitude:-8.1739059

Viseu,Oliveira De Frades,Pinheiro,
Latitude:-2.5270459
Longitude:-45.0866463

Viseu,Oliveira De Frades,Reigoso,
Latitude:41.7150016
Longitude:-7.9347419

Viseu,Oliveira De Frades,Ribeiradio,
Latitude:40.721651699999995
Longitude:-8.309439736274891

Viseu,Oliveira De Frades,São João Da Serra,
Latitude:40.7819368
Longitude:-8.2428298

Viseu,Oliveira De Frades,São Vicente De Lafões,
Latitude:40.716370749999996
Longit

Viseu,Satão,Silvã De Cima,
Latitude:40.7246728
Longitude:-7.676654

Viseu,Satão,Vila Longa,
Latitude:40.7488704
Longitude:-7.5893667

Viseu,Sernancelhe,Arnas,
Latitude:46.0235247
Longitude:4.7071167

Viseu,Sernancelhe,Carregal,
Latitude:40.9097381
Longitude:-7.5786636

Viseu,Sernancelhe,Chosendo,
Latitude:40.9678953
Longitude:-7.4559824

Viseu,Sernancelhe,Cunha,
Latitude:-23.07753
Longitude:-44.9567436

Viseu,Sernancelhe,Escurquela,
Latitude:41.004755
Longitude:-7.5210159

Viseu,Sernancelhe,Faia,
Latitude:46.9560195
Longitude:0.3418285

Viseu,Sernancelhe,Ferreirim,
Latitude:40.95109815
Longitude:-7.489777253637735

Viseu,Sernancelhe,Fonte Arcada,
Latitude:41.1471291
Longitude:-8.3623376

Viseu,Sernancelhe,Freixinho,
Latitude:40.9377108
Longitude:-7.5293932

Viseu,Sernancelhe,Granjal,
Latitude:40.8880426
Longitude:-7.5303027

Viseu,Sernancelhe,Lamosa,
Latitude:40.8866865
Longitude:-7.6216616

Viseu,Sernancelhe,Macieira,
Latitude:-26.8541639
Longitude:-51.3763845

Viseu,Sernancelhe,Penso

Viseu,Vouzela,Cambra,
Latitude:41.198139
Longitude:-76.3057705

Viseu,Vouzela,Campia,
Latitude:44.8626615
Longitude:21.4118186

Viseu,Vouzela,Carvalhal De Vermilhas,
Latitude:40.6523247
Longitude:-8.144778747357723

Viseu,Vouzela,Fataunços,
Latitude:40.7255146
Longitude:-8.0802739

Viseu,Vouzela,Figueiredo Das Donas,
Latitude:40.7326992
Longitude:-8.0468864

Viseu,Vouzela,Fornelo Do Monte,
Latitude:40.6651997
Longitude:-8.1013482

Viseu,Vouzela,Paços De Vilharigues,
Latitude:40.7121764
Longitude:-8.1288213

Viseu,Vouzela,Queirã,
Latitude:40.6887316
Longitude:-8.048031690365686

Viseu,Vouzela,São Miguel Do Mato,
Latitude:40.9652253
Longitude:-8.397602

Viseu,Vouzela,Ventosa,
Latitude:42.4042098
Longitude:-2.6263385

Viseu,Vouzela,Vouzela,
Latitude:40.6754515
Longitude:-8.148433712602198

Horta,Corvo,Corvo,
Latitude:39.69770635
Longitude:-31.105243289910312

Horta,Horta,Horta (Angustias),
Latitude:38.528478750000005
Longitude:-28.638919872786147

Horta,Horta,Capelo,
Latitude:38.5823162
L

Ponta Delgada,Ribeira Grande,Calhetas,
Latitude:37.81039665
Longitude:-25.625072154818533

Ponta Delgada,Ribeira Grande,Ribeira Grande (Conceição),
Latitude:37.79211565
Longitude:-25.49631963913462

Ponta Delgada,Ribeira Grande,Fenais Da Ajuda,
Latitude:37.8520942
Longitude:-25.3262977

Ponta Delgada,Ribeira Grande,Lomba Da Maia,
Latitude:37.813453100000004
Longitude:-25.34288745949946

Ponta Delgada,Ribeira Grande,Maia,
Latitude:41.237345649999995
Longitude:-8.629998173364326

Ponta Delgada,Ribeira Grande,Pico Da Pedra,
Latitude:-27.0771901
Longitude:-48.6602196

Ponta Delgada,Ribeira Grande,Porto Formoso,
Latitude:15.2476629
Longitude:-23.6627213

Ponta Delgada,Ribeira Grande,Rabo De Peixe,
Latitude:37.794324
Longitude:-25.583300613044948

Ponta Delgada,Ribeira Grande,Ribeira Grande (Matriz),
Latitude:37.7994125
Longitude:-25.49043963886517

Ponta Delgada,Ribeira Grande,Ribeira Seca,
Latitude:38.598903899999996
Longitude:-27.928462076279786

Ponta Delgada,Ribeira Grande,Ribeirinha,
L

In [80]:
df_loc7['Latitude'] = lat_df_loc7
df_loc7['Longitude'] = long_df_loc7

<ipython-input-80-2d2c9db292dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc7['Latitude'] = lat_df_loc7
<ipython-input-80-2d2c9db292dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc7['Longitude'] = long_df_loc7


In [81]:
districts8 = ['Angra Do Heroismo']
filter8 = df2["District"].isin(districts8)
df_loc8 = df2[filter8]
arr_df_loc8 = df_loc8.values.tolist()
lat_df_loc8 = []
long_df_loc8 = []
for d1 in arr_df_loc8:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc8.append(dsq.latitude)
    long_df_loc8.append(dsq.longitude)

Angra Do Heroismo,Angra Do Heroismo,Altares,
Latitude:38.7673652
Longitude:-27.295313957155173

Angra Do Heroismo,Angra Do Heroismo,Doze Ribeiras,
Latitude:38.7142506
Longitude:-27.3576027

Angra Do Heroismo,Angra Do Heroismo,Feteira,
Latitude:38.6582849
Longitude:-27.149229160197482

Angra Do Heroismo,Angra Do Heroismo,Angra (Nossa Senhora Da Conceição),
Latitude:38.6560792
Longitude:-27.2147423

Angra Do Heroismo,Angra Do Heroismo,Cinco Ribeiras,
Latitude:38.6837033
Longitude:-27.3155392

Angra Do Heroismo,Angra Do Heroismo,Porto Judeu,
Latitude:38.6546047
Longitude:-27.12224

Angra Do Heroismo,Angra Do Heroismo,Raminho,
Latitude:38.7711619
Longitude:-27.336179582100968

Angra Do Heroismo,Angra Do Heroismo,Ribeirinha,
Latitude:38.66562925
Longitude:-27.175938820373517

Angra Do Heroismo,Angra Do Heroismo,Santa Barbara,
Latitude:34.4221319
Longitude:-119.7026673

Angra Do Heroismo,Angra Do Heroismo,Angra (Santa Luzia),
Latitude:38.6647947
Longitude:-27.223128988519505

Angra Do Herois

In [82]:
df_loc8['Latitude'] = lat_df_loc8
df_loc8['Longitude'] = long_df_loc8

<ipython-input-82-3588ae09fe1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc8['Latitude'] = lat_df_loc8
<ipython-input-82-3588ae09fe1a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc8['Longitude'] = long_df_loc8


### Aveiro e Braga

In [83]:
districts9 = ['Aveiro', 'Braga']
filter9= df2["District"].isin(districts9)
df_loc9 = df2[filter9]
arr_df_loc9 = df_loc9.values.tolist()
lat_df_loc9 = []
long_df_loc9 = []
for d1 in arr_df_loc9:
    for v in d1:
        print(v, end = ",")
        dsq = geolocator.geocode(v)
    print('\nLatitude:{}'.format(dsq.latitude))
    print('Longitude:{}\n'.format(dsq.longitude))
    lat_df_loc9.append(dsq.latitude)
    long_df_loc9.append(dsq.longitude)

Aveiro,Agueda,Agadão,
Latitude:40.5411962
Longitude:-8.3156729

Aveiro,Agueda,Aguada De Baixo,
Latitude:40.510466
Longitude:-8.4519582

Aveiro,Agueda,Aguada De Cima,
Latitude:40.5169467
Longitude:-8.4158179

Aveiro,Agueda,Agueda,
Latitude:40.5785006
Longitude:-8.4438017

Aveiro,Agueda,Barro,
Latitude:-17.0568696
Longitude:-64.9912286

Aveiro,Agueda,Belazaima Do Chão,
Latitude:40.5329228
Longitude:-8.3660695

Aveiro,Agueda,Castanheira Do Vouga,
Latitude:40.589095
Longitude:-8.3603396

Aveiro,Agueda,Espinhel,
Latitude:40.5682765
Longitude:-8.4909913

Aveiro,Agueda,Fermentelos,
Latitude:40.5691209
Longitude:-8.5258311

Aveiro,Agueda,Lamas Do Vouga,
Latitude:40.6349709
Longitude:-8.4716592

Aveiro,Agueda,Macieira De Alcoba,
Latitude:40.620827
Longitude:-8.2741849

Aveiro,Agueda,Macinhata Do Vouga,
Latitude:40.6553986
Longitude:-8.4535347

Aveiro,Agueda,Ois Da Ribeira,
Latitude:40.5823855
Longitude:-8.5036433

Aveiro,Agueda,Prestimo,
Latitude:40.6252187
Longitude:-8.3360603

Aveiro,Agueda,R

Aveiro,Santa Maria Da Feira,Nogueira Da Regedoura,
Latitude:41.0088679
Longitude:-8.584363

Aveiro,Santa Maria Da Feira,São Paio De Oleiros,
Latitude:40.9908653
Longitude:-8.5940565

Aveiro,Santa Maria Da Feira,Paços De Brandão,
Latitude:40.9738702
Longitude:-8.5915442

Aveiro,Santa Maria Da Feira,Pigeiros,
Latitude:40.9477017
Longitude:-8.4849007

Aveiro,Santa Maria Da Feira,Rio Meão,
Latitude:40.9581668
Longitude:-8.5826133

Aveiro,Santa Maria Da Feira,Romariz,
Latitude:40.9446999
Longitude:-8.45152492108832

Aveiro,Santa Maria Da Feira,Sanfins,
Latitude:40.9264836
Longitude:-8.5254016

Aveiro,Santa Maria Da Feira,Sanguedo,
Latitude:41.0083552
Longitude:-8.5188259

Aveiro,Santa Maria Da Feira,Santa Maria De Lamas,
Latitude:40.9794526
Longitude:-8.5715712

Aveiro,Santa Maria Da Feira,São João De Ver,
Latitude:40.954172
Longitude:-8.5534047

Aveiro,Santa Maria Da Feira,Caldas De São Jorge,
Latitude:40.9663636
Longitude:-8.5044593

Aveiro,Santa Maria Da Feira,Souto,
Latitude:40.888651
L

Braga,Amares,Portela,
Latitude:41.3008566
Longitude:-8.5487452

Braga,Amares,Prozelo,
Latitude:41.8659112
Longitude:-8.4389934

Braga,Amares,Rendufe,
Latitude:41.4782335
Longitude:-8.2318882

Braga,Amares,Bouro (Santa Maria),
Latitude:41.66074855
Longitude:-8.270479064719499

Braga,Amares,Bouro (Santa Marta),
Latitude:41.6572815
Longitude:-8.2902342

Braga,Amares,Sequeiros,
Latitude:42.4546385
Longitude:-7.2354304

Braga,Amares,Seramil,
Latitude:41.6743415
Longitude:-8.304985734396638

Braga,Amares,Torre,
Latitude:40.5887189
Longitude:-4.9658282

Braga,Amares,Vilela,
Latitude:41.2434728
Longitude:-8.3802017

Braga,Barcelos,Abade De Neiva,
Latitude:41.556947
Longitude:-8.6409976

Braga,Barcelos,Aborim,
Latitude:41.6098323
Longitude:-8.6365965

Braga,Barcelos,Adães,
Latitude:41.5247198
Longitude:-8.5619714

Braga,Barcelos,Aguiar,
Latitude:41.6265225
Longitude:-8.6540794

Braga,Barcelos,Airo,
Latitude:-4.4766388
Longitude:-79.3902404

Braga,Barcelos,Aldreu,
Latitude:41.6088266
Longitude:-

Braga,Braga,Lomar,
Latitude:41.5300071
Longitude:-8.4275552

Braga,Braga,Braga (Maximinos),
Latitude:41.5435825
Longitude:-8.4332246

Braga,Braga,Mire De Tibães,
Latitude:41.5562191
Longitude:-8.4789533

Braga,Braga,Morreira,
Latitude:41.494231
Longitude:-8.4082282

Braga,Braga,Navarra,
Latitude:42.61254875
Longitude:-1.830787673361753

Braga,Braga,Nogueira,
Latitude:41.772505
Longitude:-6.8153638

Braga,Braga,Nogueiro,
Latitude:41.5513768
Longitude:-8.3876444

Braga,Braga,Padim Da Graça,
Latitude:41.5651616
Longitude:-8.4861434

Braga,Braga,Palmeira,
Latitude:-27.9010519
Longitude:-53.3135927

Braga,Braga,Panoias,
Latitude:41.5743023
Longitude:-8.46583193632081

Braga,Braga,Parada De Tibães,
Latitude:41.5623067
Longitude:-8.4636452

Braga,Braga,Pedralva,
Latitude:-22.2428119
Longitude:-45.4659922

Braga,Braga,Pousada,
Latitude:43.009354200000004
Longitude:-8.009811269855104

Braga,Braga,Priscos,
Latitude:41.4981123
Longitude:-8.4779317

Braga,Braga,Real,
Latitude:29.7949383
Longitude:

Braga,Fafe,Regadas,
Latitude:43.0502
Longitude:0.723743

Braga,Fafe,Revelhe,
Latitude:41.486026
Longitude:-8.1571894

Braga,Fafe,Ribeiros,
Latitude:-21.97776475
Longitude:-45.579699966094005

Braga,Fafe,Arões (Santa Cristina),
Latitude:41.443371
Longitude:-8.2153415

Braga,Fafe,Silvares (São Clemente),
Latitude:41.4183808
Longitude:-8.160046

Braga,Fafe,São Gens,
Latitude:39.5975057
Longitude:-7.9786378

Braga,Fafe,Silvares (São Martinho),
Latitude:41.4080255
Longitude:-8.154778

Braga,Fafe,Arões (São Romão),
Latitude:41.45615
Longitude:-8.2175346

Braga,Fafe,Seidões,
Latitude:41.4073018
Longitude:-8.1205576

Braga,Fafe,Serafão,
Latitude:41.5293932
Longitude:-8.2027837

Braga,Fafe,Travassos,
Latitude:41.5754553
Longitude:-8.2023537

Braga,Fafe,Varzea Cova,
Latitude:41.5085893
Longitude:-8.0696939

Braga,Fafe,Vila Cova,
Latitude:41.3159721
Longitude:-7.8568941

Braga,Fafe,Vinhos,
Latitude:41.1007631
Longitude:-7.9522348

Braga,Guimarães,Aldão,
Latitude:41.4644195
Longitude:-8.2737301

B

Braga,Povoa De Lanhoso,Taide,
Latitude:41.5578325
Longitude:-8.2297605

Braga,Povoa De Lanhoso,Travassos,
Latitude:41.5754553
Longitude:-8.2023537

Braga,Povoa De Lanhoso,Verim,
Latitude:41.6335702
Longitude:-8.3037858

Braga,Povoa De Lanhoso,Vilela,
Latitude:41.2434728
Longitude:-8.3802017

Braga,Terras De Bouro,Balança,
Latitude:-3.6719328
Longitude:-39.6584847

Braga,Terras De Bouro,Brufe,
Latitude:41.4130747
Longitude:-8.5394855

Braga,Terras De Bouro,Campo Do Geres,
Latitude:41.75794965
Longitude:-8.200511491235382

Braga,Terras De Bouro,Carvalheira,
Latitude:41.4701006
Longitude:-8.5144047

Braga,Terras De Bouro,Chamoim,
Latitude:41.7349542
Longitude:-8.2705398

Braga,Terras De Bouro,Chorense,
Latitude:41.7064776
Longitude:-8.3036825

Braga,Terras De Bouro,Cibões,
Latitude:41.7588701
Longitude:-8.2884131

Braga,Terras De Bouro,Covide,
Latitude:41.7386963
Longitude:-8.2145413

Braga,Terras De Bouro,Gondoriz,
Latitude:41.905206
Longitude:-8.4075995

Braga,Terras De Bouro,Moimenta,


Braga,Vila Verde,Esqueiros,
Latitude:41.6623721
Longitude:-8.4515055

Braga,Vila Verde,Freiriz,
Latitude:41.64336095
Longitude:-8.508945039820423

Braga,Vila Verde,Geme,
Latitude:45.4885123
Longitude:13.782867

Braga,Vila Verde,Goães,
Latitude:43.8149643
Longitude:0.9635264

Braga,Vila Verde,Godinhaços,
Latitude:41.7121675
Longitude:-8.4589219

Braga,Vila Verde,Gomide,
Latitude:41.7160606
Longitude:-8.3786604

Braga,Vila Verde,Gondiães,
Latitude:41.5932621
Longitude:-7.8439052

Braga,Vila Verde,Gondomar,
Latitude:41.1398199
Longitude:-8.5326269

Braga,Vila Verde,Laje,
Latitude:-13.1809956
Longitude:-39.4222968

Braga,Vila Verde,Lanhas,
Latitude:43.8941311
Longitude:5.1148619

Braga,Vila Verde,Loureira,
Latitude:41.633169
Longitude:-8.4237379

Braga,Vila Verde,Marrancos,
Latitude:41.6666136
Longitude:-8.5179288

Braga,Vila Verde,Mos,
Latitude:42.2030958
Longitude:-8.6085418

Braga,Vila Verde,Moure,
Latitude:41.5013821
Longitude:-8.5552403

Braga,Vila Verde,Nevogilde,
Latitude:41.259913


In [84]:
df_loc9['Latitude'] = lat_df_loc9
df_loc9['Longitude'] = long_df_loc9

<ipython-input-84-de91879415ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc9['Latitude'] = lat_df_loc9
<ipython-input-84-de91879415ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loc9['Longitude'] = long_df_loc9


In [85]:
df3 = df_loc2.append([df_loc1, df_loc5, df_loc4, df_porto, df_loc6, df_loc7, df_loc8, df_loc9],ignore_index = True)

In [88]:
df3.sort_values(by=['District'], inplace=True)
df4 = df3.reset_index(drop=True)
df4.head()

,District,County,Borough,Latitude,Longitude
0,Angra Do Heroismo,Calheta (Açores),Santo Antão,17.055529,-25.172305
1,Angra Do Heroismo,Santa Cruz Da Graciosa,Guadalupe,16.230510,-61.687126
2,Angra Do Heroismo,Santa Cruz Da Graciosa,Luz,-23.535122,-46.635393
3,Angra Do Heroismo,Santa Cruz Da Graciosa,Santa Cruz Da Graciosa,39.052890,-28.012979
4,Angra Do Heroismo,Santa Cruz Da Graciosa,São Mateus,-18.715634,-39.848256


Store the new dataset into csv for security

In [89]:
data = df4.to_csv('PortugalBorough.csv', index = True, encoding = 'utf-8-sig')

In [90]:
df4.District.value_counts()

Braga                519
Porto                389
Viseu                372
Guarda               335
Braganca             298
Viana Do Castelo     290
Vila Real            268
Lisboa               235
Coimbra              209
Aveiro               208
Santarem             192
Castelo Branco       160
Leiria               149
Beja                 100
Evora                 93
Portalegre            86
Faro                  84
Setubal               82
Ponta Delgada         71
Funchal               55
Angra Do Heroismo     45
Horta                 42
Name: District, dtype: int64

### Visualizing the neighbourhoods from Lisboa and Setubal

In [91]:
latitudes = df4.Latitude.values
longitudes = df4.Longitude.values
lisbon_df = df4[df4['District'].isin(['Lisboa'])]
setubal_df = df4[df4['District'].isin(['Setubal'])]

In [92]:
address = 'Lisboa, Portugal'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.7077507 -9.1365919


### Lisbon neighbors

In [93]:
# create map of Toronto using latitude and longitude values
lisbon_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map based on coordinates of postcode
for lat, lng, borough in zip(lisbon_df['Latitude'], lisbon_df['Longitude'], lisbon_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(lisbon_map)  
    
lisbon_map

In [339]:
address2 = 'Costa da Caparica, Portugal'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address)
latitude2 = location.latitude
longitude2 = location.longitude
print(latitude2, longitude2)

38.7077507 -9.1365919


In [95]:
stb_map = folium.Map(location=[latitude2, longitude2], zoom_start=10)
# add markers to map based on coordinates of postcode
for lat, lng, borough in zip(setubal_df['Latitude'], setubal_df['Longitude'], setubal_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(stb_map)  
    
stb_map

# Capstone Project - Week 2

#### One of the most famous places in Setubal District is Costa da Caparica which is very close to Lisbon

Extract the coordinates from Costa da Caparica

In [342]:
df4[df4.Borough == 'Costa Da Caparica']

,District,County,Borough,Latitude,Longitude
3318,Setubal,Almada,Costa Da Caparica,38.644517,-9.233607


In [365]:
CLIENT_ID = 'U0KVXUKLDIMOPFGFKEGWTIYYDIFW4T1SERP50J25KQN3GNYH' # your Foursquare ID
CLIENT_SECRET = 'AWXU2HYVF1PKWQRIHH5YLUHDSKREEFDBXAPPEUCUZMBOPNTM' # your Foursquare Secret
ACCESS_TOKEN = 'R5NGW11OKBUY0FOVGS1WVIENVVJ0THKWSZXT3RCUWN2C50MP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 500
radius = 10000
latitudes = setubal_df.Latitude.values
longitudes = setubal_df.Longitude.values

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            lat, 
            lng,
            ACCESS_TOKEN,
            VERSION,
            radius,
            LIMIT
            )
results = requests.get(url).json()['response']['venues']
dataframe = json_normalize(results)
dataframe

<ipython-input-365-a29f878b6c8a>:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(results)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,4e6c9251d22d0f5096235c9f,A Nutritiva,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622900401,False,"Pr. da Liberdade, 2",38.644059,-9.233719,"[{'label': 'display', 'lat': 38.64405922156645...",51,2825-370,PT,Costa da Caparica,Setúbal,Portugal,"[Pr. da Liberdade, 2, 2825-370 Costa da Caparica]",NaN,NaN,NaN
1,4e19cd9cfa761946459df8c1,Centro Comercial Pescador,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1622900401,False,"Praça da Liberdade, 17",38.644484,-9.233871,"[{'label': 'display', 'lat': 38.64448377213313...",23,2825-355,PT,Costa da Caparica,Setúbal,Portugal,"[Praça da Liberdade, 17, 2825-355 Costa da Cap...",NaN,NaN,NaN
2,5f0cb9fba718601ded92ccce,Do Nails And Lashes,"[{'id': '54541900498ea6ccd0202697', 'name': 'H...",v-1622900401,False,Centro Comercial Pescador - Loja 0.22,38.644345,-9.233593,"[{'label': 'display', 'lat': 38.644345, 'lng':...",19,2825-355,PT,Costa da Caparica,Setúbal,Portugal,"[Centro Comercial Pescador - Loja 0.22, 2825-3...",NaN,NaN,NaN
3,4ebf89056da15b4ddee93dd0,Caixa Geral de Depósitos (Costa da Caparica),"[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",v-1622900401,False,"Praça da Liberdade, 18",38.644600,-9.233585,"[{'label': 'display', 'lat': 38.64459970730071...",9,2825-355,PT,Almada,Setúbal,Portugal,"[Praça da Liberdade, 18 (Av. D. Sebastião), 28...",Av. D. Sebastião,NaN,NaN
4,4eb3a7949a52ad1e3e270643,Praça da Liberdade,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1622900401,False,Praça da Liberdade,38.644277,-9.233778,"[{'label': 'display', 'lat': 38.64427709227535...",30,2825,PT,Costa da Caparica,Setúbal,Portugal,"[Praça da Liberdade, 2825 Costa da Caparica]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,51d01165498e6594b97b4264,Saltarico,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1622900401,False,NaN,38.642388,-9.233176,"[{'label': 'display', 'lat': 38.642388, 'lng':...",239,NaN,PT,NaN,NaN,Portugal,NaN,NaN,NaN,NaN
113,521e3164498e3488b1b110f8,À Flor da Pele,"[{'id': '4f04aa0c2fb6e1c99f3db0b8', 'name': 'N...",v-1622900401,False,"Rua Sant'Ana à Lapa, 33A",38.643926,-9.234525,"[{'label': 'display', 'lat': 38.64392600017439...",103,1200,PT,Lisboa,Lisboa,Portugal,"[Rua Sant'Ana à Lapa, 33A, 1200 Lisboa]",NaN,NaN,NaN
114,500abfd4e4b02a36e9f0c0ed,Barbearia Tradicional Soares,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",v-1622900401,False,R. Padre Baltazar,38.643271,-9.232377,"[{'label': 'display', 'lat': 38.64327078284363...",175,2825,PT,Costa da Caparica,Setúbal,Portugal,[R. Padre Baltazar (Av. Dr. Aresta Branco & R....,Av. Dr. Aresta Branco & R. Rebelo da Silva,NaN,NaN
115,50485a73e4b00f91b9b68db6,Centro de Medicina Naval,"[{'id': '4bf58dd8d48988d177941735', 'name': 'D...",v-1622900401,False,NaN,38.643529,-9.231782,"[{'label': 'display', 'lat': 38.64352908541003...",193,NaN,PT,NaN,NaN,Portugal,NaN,NaN,NaN,NaN


In [372]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
costa_df = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
costa_df['categories'] = costa_df.apply(get_category_type, axis=1)

# clean column names by keeping only last term
costa_df.columns = [column.split('.')[-1] for column in costa_df.columns]

costa_df

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,A Nutritiva,Coffee Shop,"Pr. da Liberdade, 2",38.644059,-9.233719,"[{'label': 'display', 'lat': 38.64405922156645...",51,2825-370,PT,Costa da Caparica,Setúbal,Portugal,"[Pr. da Liberdade, 2, 2825-370 Costa da Caparica]",NaN,NaN,4e6c9251d22d0f5096235c9f
1,Centro Comercial Pescador,Shopping Mall,"Praça da Liberdade, 17",38.644484,-9.233871,"[{'label': 'display', 'lat': 38.64448377213313...",23,2825-355,PT,Costa da Caparica,Setúbal,Portugal,"[Praça da Liberdade, 17, 2825-355 Costa da Cap...",NaN,NaN,4e19cd9cfa761946459df8c1
2,Do Nails And Lashes,Health & Beauty Service,Centro Comercial Pescador - Loja 0.22,38.644345,-9.233593,"[{'label': 'display', 'lat': 38.644345, 'lng':...",19,2825-355,PT,Costa da Caparica,Setúbal,Portugal,"[Centro Comercial Pescador - Loja 0.22, 2825-3...",NaN,NaN,5f0cb9fba718601ded92ccce
3,Caixa Geral de Depósitos (Costa da Caparica),Bank,"Praça da Liberdade, 18",38.644600,-9.233585,"[{'label': 'display', 'lat': 38.64459970730071...",9,2825-355,PT,Almada,Setúbal,Portugal,"[Praça da Liberdade, 18 (Av. D. Sebastião), 28...",Av. D. Sebastião,NaN,4ebf89056da15b4ddee93dd0
4,Praça da Liberdade,Plaza,Praça da Liberdade,38.644277,-9.233778,"[{'label': 'display', 'lat': 38.64427709227535...",30,2825,PT,Costa da Caparica,Setúbal,Portugal,"[Praça da Liberdade, 2825 Costa da Caparica]",NaN,NaN,4eb3a7949a52ad1e3e270643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,Saltarico,Brewery,NaN,38.642388,-9.233176,"[{'label': 'display', 'lat': 38.642388, 'lng':...",239,NaN,PT,NaN,NaN,Portugal,NaN,NaN,NaN,51d01165498e6594b97b4264
113,À Flor da Pele,Nail Salon,"Rua Sant'Ana à Lapa, 33A",38.643926,-9.234525,"[{'label': 'display', 'lat': 38.64392600017439...",103,1200,PT,Lisboa,Lisboa,Portugal,"[Rua Sant'Ana à Lapa, 33A, 1200 Lisboa]",NaN,NaN,521e3164498e3488b1b110f8
114,Barbearia Tradicional Soares,Salon / Barbershop,R. Padre Baltazar,38.643271,-9.232377,"[{'label': 'display', 'lat': 38.64327078284363...",175,2825,PT,Costa da Caparica,Setúbal,Portugal,[R. Padre Baltazar (Av. Dr. Aresta Branco & R....,Av. Dr. Aresta Branco & R. Rebelo da Silva,NaN,500abfd4e4b02a36e9f0c0ed
115,Centro de Medicina Naval,Doctor's Office,NaN,38.643529,-9.231782,"[{'label': 'display', 'lat': 38.64352908541003...",193,NaN,PT,NaN,NaN,Portugal,NaN,NaN,NaN,50485a73e4b00f91b9b68db6


### Grouping Venue categories

In [394]:
costa_filtered = costa_df.iloc[:,:5]
costa_filtered['categories'].value_counts()

Coffee Shop              6
Nail Salon               6
Bank                     5
Burger Joint             4
Portuguese Restaurant    4
                        ..
Medical Center           1
Hostel                   1
Vacation Rental          1
Bed & Breakfast          1
Italian Restaurant       1
Name: categories, Length: 73, dtype: int64

#### For clustering, it's necessary One Hot Encoding

In [395]:
costa_filtered_dummies = pd.get_dummies(costa_filtered[['categories']], prefix="", prefix_sep="")
costa_filtered_dummies

,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Boutique,Brazilian Restaurant,Brewery,Burger Joint,...,Spanish Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Theater,Vacation Rental,Video Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [397]:
costa_filtered_dummies['Neighborhood'] = 'Costa da Caparica'

fixed_columns = [costa_filtered_dummies.columns[-1]] + list(costa_filtered_dummies.columns[:-1])
costa_filtered_dummies = costa_filtered_dummies[fixed_columns]

costa_filtered_dummies.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Boutique,Brazilian Restaurant,Brewery,...,Spanish Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Theater,Vacation Rental,Video Store,Yoga Studio
0,Costa da Caparica,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Costa da Caparica,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Costa da Caparica,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Costa da Caparica,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Costa da Caparica,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [398]:
costa_g = costa_filtered_dummies.groupby('Neighborhood').mean().reset_index()
costa_g.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Boutique,Brazilian Restaurant,Brewery,...,Spanish Restaurant,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Theater,Vacation Rental,Video Store,Yoga Studio
0,Costa da Caparica,0.008547,0.017094,0.008547,0.042735,0.008547,0.008547,0.008547,0.008547,0.008547,...,0.008547,0.025641,0.008547,0.008547,0.008547,0.008547,0.008547,0.008547,0.008547,0.008547


In [399]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [404]:
top = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [405]:
# create a new dataframe for London
costa_sorted = pd.DataFrame(columns=columns)
costa_sorted['Neighborhood'] = costa_g['Neighborhood']

for ind in np.arange(costa_g.shape[0]):
    costa_sorted.iloc[ind, 1:] = return_most_common_venues(costa_g.iloc[ind, :], top)

costa_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Costa da Caparica,Coffee Shop,Nail Salon,Bank,Portuguese Restaurant,Butcher,Burger Joint,Restaurant,Supermarket,Miscellaneous Shop,Newsstand


### Clustering with K Means

In [419]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics


In [437]:
costa_cluster = costa_filtered_dummies.drop('Neighborhood',1)

In [438]:
clusters = 3
costa_KMeans = KMeans(n_clusters=clusters, random_state=0).fit(costa_cluster)
costa_KMeans

KMeans(n_clusters=3, random_state=0)

In [439]:
costa_KMeans.labels_

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 2, 0, 0, 0])

In [441]:
costa_cluster['Cluster Labels'] =  costa_KMeans.labels_

In [449]:
costa_cluster

,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Boutique,Brazilian Restaurant,Brewery,Burger Joint,...,Supermarket,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Theater,Vacation Rental,Video Store,Yoga Studio,Cluster Labels
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [448]:
costa_filtered

,name,categories,address,lat,lng
0,A Nutritiva,Coffee Shop,"Pr. da Liberdade, 2",38.644059,-9.233719
1,Centro Comercial Pescador,Shopping Mall,"Praça da Liberdade, 17",38.644484,-9.233871
2,Do Nails And Lashes,Health & Beauty Service,Centro Comercial Pescador - Loja 0.22,38.644345,-9.233593
3,Caixa Geral de Depósitos (Costa da Caparica),Bank,"Praça da Liberdade, 18",38.644600,-9.233585
4,Praça da Liberdade,Plaza,Praça da Liberdade,38.644277,-9.233778
...,...,...,...,...,...
112,Saltarico,Brewery,NaN,38.642388,-9.233176
113,À Flor da Pele,Nail Salon,"Rua Sant'Ana à Lapa, 33A",38.643926,-9.234525
114,Barbearia Tradicional Soares,Salon / Barbershop,R. Padre Baltazar,38.643271,-9.232377
115,Centro de Medicina Naval,Doctor's Office,NaN,38.643529,-9.231782


In [451]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [454]:
costa_map = folium.Map(location=[38.644517, -9.233607], zoom_start=12)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(costa_filtered['lat'], costa_filtered['lng'], costa_filtered['name'], costa_cluster['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(costa_map)
        
costa_map

### Lisbon venues

Martires is a central point in Lisbon Area 

In [468]:
df4[df4.Borough == 'Santos-O-Velho']

,District,County,Borough,Latitude,Longitude
2327,Lisboa,Lisboa,Santos-O-Velho,38.707323,-9.15738


In [469]:
CLIENT_ID = 'U0KVXUKLDIMOPFGFKEGWTIYYDIFW4T1SERP50J25KQN3GNYH' # your Foursquare ID
CLIENT_SECRET = 'AWXU2HYVF1PKWQRIHH5YLUHDSKREEFDBXAPPEUCUZMBOPNTM' # your Foursquare Secret
ACCESS_TOKEN = 'R5NGW11OKBUY0FOVGS1WVIENVVJ0THKWSZXT3RCUWN2C50MP' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 500
radius = 10000
lat2 = 38.707323
lng2 = -9.15738
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            lat2, 
            lng2,
            ACCESS_TOKEN,
            VERSION,
            radius,
            LIMIT
            )
results = requests.get(url).json()['response']['venues']
dataframe2 = json_normalize(results)
dataframe2

<ipython-input-469-59c498487023>:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe2 = json_normalize(results)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet,location.neighborhood
0,5912b5326bdee66b586fdd63,Mercearia da Mila,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1622906516,False,Rua de Santos-o-velho 38,38.707218,-9.157016,"[{'label': 'display', 'lat': 38.70721815071557...",33,1200-812,PT,Lisboa,Lisboa,Portugal,"[Rua de Santos-o-velho 38, 1200-812 Lisboa]",NaN,NaN,NaN
1,54de4d17498edd530584ed18,Mercado de Santos,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",v-1622906516,False,NaN,38.707883,-9.155829,"[{'label': 'display', 'lat': 38.70788270613389...",148,NaN,PT,NaN,NaN,Portugal,NaN,NaN,NaN,NaN
2,5062525ee4b02b3bbf6ff099,Taverna do Cura,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1622906516,False,"Rua Do Cura, 1",38.707482,-9.157402,"[{'label': 'display', 'lat': 38.70748165250612...",17,NaN,PT,Lisboa,Lisboa,Portugal,"[Rua Do Cura, 1, Lisboa]",NaN,NaN,NaN
3,56bb7439498eabc25631792b,Il Covo,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622906516,False,"Rua do Cura 1, 3A",38.707517,-9.157408,"[{'label': 'display', 'lat': 38.70751745855340...",21,NaN,PT,Lisboa,Lisboa,Portugal,"[Rua do Cura 1, 3A, Lisboa]",NaN,NaN,NaN
4,4fb51db5e4b015cb224a381a,Largo da Igreja,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1622906516,False,Igreja De Santos-o-velho,38.706783,-9.157942,"[{'label': 'display', 'lat': 38.70678289344019...",77,NaN,PT,Lisboa,Lisboa,Portugal,"[Igreja De Santos-o-velho, Lisboa]",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,570eb23f498e8feaaf2935a1,Alfaiataria,[],v-1622906516,False,Rua São João da Mata 67,38.707851,-9.158661,"[{'label': 'display', 'lat': 38.707851, 'lng':...",125,1200,PT,Lisboa,Lisboa,Portugal,"[Rua São João da Mata 67, 1200 Lisboa]",NaN,NaN,NaN
118,5c864d7453159300392fef1e,Mamasan,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1622906516,False,"Calçada Marquês de Abrantes, 140",38.707634,-9.155801,"[{'label': 'display', 'lat': 38.707634, 'lng':...",141,1200-720,PT,Lisboa,Lisboa,Portugal,"[Calçada Marquês de Abrantes, 140, 1200-720 Li...",NaN,NaN,NaN
119,50f20b31e4b070ff05de13a5,ZERO,"[{'id': '4bf58dd8d48988d11f941735', 'name': 'N...",v-1622906516,False,"Av. 24 Julho, 68",38.705950,-9.157558,"[{'label': 'display', 'lat': 38.70594976091142...",153,NaN,PT,Lisboa,Lisboa,Portugal,"[Av. 24 Julho, 68, Lisboa]",NaN,NaN,NaN
120,5478c9f2498eb0f50cdc4941,"Caco, o original","[{'id': '4def73e84765ae376e57713a', 'name': 'P...",v-1622906516,False,NaN,38.707085,-9.156148,"[{'label': 'display', 'lat': 38.70708492172292...",110,NaN,PT,NaN,NaN,Portugal,NaN,NaN,NaN,NaN


In [474]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
chiado_df = dataframe2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# filter the category for each row
chiado_df['categories'] = chiado_df.apply(get_category_type, axis=1)

# clean column names by keeping only last term
chiado_df.columns = [column.split('.')[-1] for column in chiado_df.columns]

chiado_df
chiado_filtered = chiado_df.iloc[:,:5]

,name,categories,address,lat,lng
0,Mercearia da Mila,Deli / Bodega,Rua de Santos-o-velho 38,38.707218,-9.157016
1,Mercado de Santos,Market,NaN,38.707883,-9.155829
2,Taverna do Cura,Bar,"Rua Do Cura, 1",38.707482,-9.157402
3,Il Covo,Italian Restaurant,"Rua do Cura 1, 3A",38.707517,-9.157408
4,Largo da Igreja,Plaza,Igreja De Santos-o-velho,38.706783,-9.157942
...,...,...,...,...,...
117,Alfaiataria,None,Rua São João da Mata 67,38.707851,-9.158661
118,Mamasan,Japanese Restaurant,"Calçada Marquês de Abrantes, 140",38.707634,-9.155801
119,ZERO,Nightclub,"Av. 24 Julho, 68",38.705950,-9.157558
120,"Caco, o original",Portuguese Restaurant,NaN,38.707085,-9.156148


In [475]:
chiado_filtered['categories'].value_counts()

Nightclub                                   12
Restaurant                                  11
Portuguese Restaurant                       10
Bar                                          8
Office                                       6
Café                                         5
Plaza                                        3
Building                                     3
Hotel                                        3
Indian Restaurant                            3
Residential Building (Apartment / Condo)     3
Mediterranean Restaurant                     2
Other Nightlife                              2
Breakfast Spot                               2
Salon / Barbershop                           1
Hostel                                       1
Tapas Restaurant                             1
Business Center                              1
Market                                       1
Bus Station                                  1
General Entertainment                        1
Tattoo Parlor

In [476]:
lisbon_filtered_dummies = pd.get_dummies(chiado_filtered[['categories']], prefix="", prefix_sep="")
lisbon_filtered_dummies

,Advertising Agency,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Boutique,Breakfast Spot,Building,...,Spanish Restaurant,Steakhouse,Surf Spot,Tapas Restaurant,Tattoo Parlor,Theme Restaurant,Veterinarian,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [480]:
lisbon_filtered_dummies['Neighborhood'] = 'Chiado'

fixed_columns = [lisbon_filtered_dummies.columns[-1]] + list(lisbon_filtered_dummies.columns[:-1])
lisbon_filtered_dummies = lisbon_filtered_dummies[fixed_columns]

lisbon_filtered_dummies.head()

,Yoga Studio,Neighborhood,Advertising Agency,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Boutique,...,Sandwich Place,Spanish Restaurant,Steakhouse,Surf Spot,Tapas Restaurant,Tattoo Parlor,Theme Restaurant,Veterinarian,Wine Bar,Wine Shop
0,0,Chiado,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Chiado,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Chiado,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,Chiado,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,Chiado,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [481]:
# create a new dataframe for London
chiado_g = lisbon_filtered_dummies.groupby('Neighborhood').mean().reset_index()
chiado_g.head()

,Neighborhood,Yoga Studio,Advertising Agency,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Boutique,...,Sandwich Place,Spanish Restaurant,Steakhouse,Surf Spot,Tapas Restaurant,Tattoo Parlor,Theme Restaurant,Veterinarian,Wine Bar,Wine Shop
0,Chiado,0.008197,0.008197,0.008197,0.008197,0.008197,0.065574,0.008197,0.008197,0.008197,...,0.008197,0.008197,0.008197,0.008197,0.008197,0.008197,0.008197,0.008197,0.008197,0.008197


In [483]:
# create a new dataframe for London
santos_sorted = pd.DataFrame(columns=columns)
santos_sorted['Neighborhood'] = chiado_g['Neighborhood']

for ind in np.arange(costa_g.shape[0]):
    santos_sorted.iloc[ind, 1:] = return_most_common_venues(chiado_g.iloc[ind, :], top)

santos_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chiado,Nightclub,Restaurant,Portuguese Restaurant,Bar,Office,Café,Residential Building (Apartment / Condo),Plaza,Building,Hotel


### K Means

In [484]:
santos_cluster = lisbon_filtered_dummies.drop('Neighborhood',1)

In [487]:
santos_KMeans = KMeans(n_clusters=clusters, random_state=0).fit(santos_cluster)
santos_KMeans

KMeans(n_clusters=3, random_state=0)

In [488]:
santos_KMeans.labels_
santos_cluster['Cluster Labels'] =  santos_KMeans.labels_

In [489]:
santos_cluster

,Yoga Studio,Advertising Agency,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Boutique,Breakfast Spot,...,Spanish Restaurant,Steakhouse,Surf Spot,Tapas Restaurant,Tattoo Parlor,Theme Restaurant,Veterinarian,Wine Bar,Wine Shop,Cluster Labels
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            lat, 
            lng,
            ACCESS_TOKEN,
            VERSION,
            radius,
            LIMIT
            )
results = requests.get(url).json()['response']['venues']
dataframe = json_normalize(results)
dataframe

### Visualizing Clusters in Santos

In [491]:

santos_map = folium.Map(location=[38.644517, -9.233607], zoom_start=12)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chiado_filtered['lat'], chiado_filtered['lng'], chiado_filtered['name'], santos_cluster['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(santos_map)
        
santos_map

### Results and Discussion

Costa da Caparica is a place with a lot of beaches, I think it's expected there are more food establishments. On the other hand, since Lisbon is a very common place that tourists chose, there are a more "life" comparing with Costa da Caparica where we can find more night clubs, restaurants and Portuguese Restaurants

Santos, is a little drop in the ocean, in Lisbon there are thousands of places. Costa da Caparica is a small beach village with great bars and great sun.

### Conclusion
I think tourists should investigate further when come to Portugal, Lisbon is great but there are really good places in District of Setubal where the prices of living drops (sometimes more than 30%) 

### Limitations 

Portugal is a small country. as expected, we don't have too much data to play with. This small exercise gave a small idea when comparing two tourists places with different purposes.

### Foursquare

It was a problem Foursquare limits the calls, in my case, I easily exceeded the rate limits established by the App, I had to wait sometimes 24h to make more API calls.